# 👥 Análisis Sociodemográfico de Desigualdad en España (2008-2023)

## Desigualdad por Edad, Sexo, Tipo de Hogar y Situación Laboral

---

### 🎯 Objetivos del Análisis

- **Identificar grupos vulnerables** por edad, sexo, tipo de hogar y situación laboral
- **Cuantificar desigualdades** mediante tasas AROPE y personas afectadas
- **Analizar evolución temporal** de vulnerabilidades (2008-2023)
- **Detectar patrones de interseccionalidad**: Cruces multidimensionales
- **Proponer políticas diferenciadas** según perfil de riesgo

---

### 📊 Datos y Metodología

**Fuentes:**
- INE - Encuesta de Condiciones de Vida (ECV)
- INE - Padrón municipal (población)
- Eurostat (metodología AROPE)

**Indicador Principal:**
- **AROPE** (At Risk of Poverty Or Social Exclusion)
  - Pobreza monetaria: Ingresos < 60% mediana
  - Deprivación material severa
  - Baja intensidad de empleo en hogar

**Período:** 2008-2023 (16 años)

---

## 1️⃣ Configuración y Carga de Datos

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
from sqlalchemy import create_engine
import urllib.parse
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['font.size'] = 10
pd.set_option('display.max_columns', None)

print("✅ Librerías importadas correctamente")

In [ ]:
# Conexión a SQL Server
SERVER = 'localhost'
DATABASE = 'desigualdad'
DRIVER = '{ODBC Driver 17 for SQL Server}'

try:
    connection_string = f'DRIVER={DRIVER};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;'
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}')
    connection = engine.connect()
    print(f"✅ Conexión exitosa a: {SERVER}/{DATABASE}")
    connection.close()
except Exception as e:
    print(f"❌ Error de conexión: {e}")

In [ ]:
# Función helper: Detectar columna de valor AROPE
def find_valor_col(df, table_name="tabla"):
    """Busca columna de valor AROPE de forma robusta"""
    if df is None or df.empty:
        return None
    
    # Buscar por nombre
    for col in df.columns:
        if any(x in col.upper() for x in ['AROPE', 'TASA', 'VALOR']):
            return col
    
    # Buscar primera columna numérica que no sea dimensión
    dimensiones = ['año', 'edad', 'sexo', 'tipo_hogar', 'situacion_laboral', 'ccaa', 'indicador', 'decil']
    for col in df.columns:
        if col.lower() not in dimensiones and df[col].dtype in ['float64', 'int64']:
            return col
    
    return None

print("✅ Funciones helper cargadas")

In [ ]:
# Cargar datos sociodemográficos
print("=" * 100)
print("CARGA DE DATOS SOCIODEMOGRÁFICOS")
print("=" * 100)

# AROPE por Edad y Sexo
df_arope_edad_sexo = pd.read_sql("SELECT * FROM INE_AROPE_Edad_Sexo ORDER BY Año, Edad, Sexo", engine)
print(f"\n✓ AROPE Edad/Sexo: {len(df_arope_edad_sexo)} registros ({df_arope_edad_sexo['Año'].min()}-{df_arope_edad_sexo['Año'].max()})")

# AROPE por Tipo de Hogar
df_arope_hogar = pd.read_sql("SELECT * FROM INE_AROPE_Hogar ORDER BY Año, Tipo_Hogar", engine)
print(f"✓ AROPE Hogar: {len(df_arope_hogar)} registros ({df_arope_hogar['Año'].min()}-{df_arope_hogar['Año'].max()})")

# AROPE por Situación Laboral
df_arope_laboral = pd.read_sql("SELECT * FROM INE_AROPE_Laboral ORDER BY Año, Situacion_Laboral", engine)
print(f"✓ AROPE Laboral: {len(df_arope_laboral)} registros ({df_arope_laboral['Año'].min()}-{df_arope_laboral['Año'].max()})")

# Población por Edad y Sexo
df_poblacion = pd.read_sql("""SELECT Año, Sexo, Edad, Poblacion 
                              FROM INE_Poblacion_Edad_Sexo 
                              WHERE Año >= 2008 ORDER BY Año, Edad, Sexo""", engine)
print(f"✓ Población: {len(df_poblacion)} registros ({df_poblacion['Año'].min()}-{df_poblacion['Año'].max()})")

# Normalizar nombres de Sexo (CRÍTICO para el cruce)
mapeo_sexo = {'Hombres': 'Hombre', 'Mujeres': 'Mujer', 'Total': 'Total'}
df_poblacion['Sexo'] = df_poblacion['Sexo'].map(mapeo_sexo)

# Excluir grupo agregado
df_poblacion = df_poblacion[df_poblacion['Edad'] != 'Todas las edades']

print("\n✅ Datos cargados y normalizados correctamente")
print("=" * 100)

---

## 2️⃣ Preparación de Datos: Mapeo y Cruce AROPE × Población

In [ ]:
# Mapeo: Población Quinquenal → Grupos AROPE
print("=" * 100)
print("MAPEO DE GRUPOS DE EDAD: Quinquenal → AROPE")
print("=" * 100)

# Mapeo de grupos quinquenales (población) a grupos AROPE (tasas vulnerabilidad)
mapeo_edad_arope = {
    '0-4': 'Menores de 16 años',
    '5-9': 'Menores de 16 años',
    '10-14': 'Menores de 16 años',
    '15-19': '16 a 29 años',
    '20-24': '16 a 29 años',
    '25-29': '16 a 29 años',
    '30-34': '30 a 44 años',
    '35-39': '30 a 44 años',
    '40-44': '30 a 44 años',
    '45-49': '45 a 64 años',
    '50-54': '45 a 64 años',
    '55-59': '45 a 64 años',
    '60-64': '45 a 64 años',
    '65-69': '65 y más años',
    '70-74': '65 y más años',
    '75-79': '65 y más años',
    '80-84': '65 y más años',
    '85-89': '65 y más años',
    '90-94': '65 y más años',
    '90+': '65 y más años',
    '95+': '65 y más años'
}

# Aplicar mapeo
df_poblacion['Edad_AROPE'] = df_poblacion['Edad'].map(mapeo_edad_arope)

# Diagnosticar edades sin mapeo
edades_sin_mapeo = df_poblacion[df_poblacion['Edad_AROPE'].isna()]['Edad'].unique()
if len(edades_sin_mapeo) > 0:
    print(f"\n⚠️  Edades SIN mapeo: {sorted(edades_sin_mapeo)}")
else:
    print(f"\n✓ Todas las edades mapeadas correctamente")

# Agrupar población por grupos AROPE
df_poblacion_arope = df_poblacion[df_poblacion['Edad_AROPE'].notna()].groupby(
    ['Año', 'Sexo', 'Edad_AROPE'], 
    as_index=False
)['Poblacion'].sum()

print(f"✓ Población agregada: {len(df_poblacion_arope)} registros")
print(f"  Periodo: {df_poblacion_arope['Año'].min()}-{df_poblacion_arope['Año'].max()}")
print(f"  Grupos edad: {df_poblacion_arope['Edad_AROPE'].nunique()}")

print("\n" + "=" * 100)

In [ ]:
# Cruce AROPE × Población
print("=" * 100)
print("CRUCE: TASAS AROPE × POBLACIÓN = PERSONAS EN RIESGO")
print("=" * 100)

# Grupos válidos (mutuamente excluyentes)
grupos_validos = ['Menores de 16 años', '16 a 29 años', '30 a 44 años', '45 a 64 años', '65 y más años']

# Detectar columna de valor AROPE
valor_col = find_valor_col(df_arope_edad_sexo, "AROPE_Edad_Sexo")

# Filtrar solo AROPE total (no componentes) y solo grupos válidos
df_arope_total = df_arope_edad_sexo[
    (df_arope_edad_sexo['Indicador'] == 'AROPE') & 
    (df_arope_edad_sexo['Edad'].isin(grupos_validos))
].copy()

# Renombrar columna Edad a Edad_AROPE para el merge
df_arope_total = df_arope_total.rename(columns={'Edad': 'Edad_AROPE'})

# Limpiar duplicados en AROPE (usar primer valor cuando hay triplicados)
df_arope_clean = df_arope_total.groupby(['Año', 'Sexo', 'Edad_AROPE'], as_index=False).agg({
    'Valor': 'first'
})

# Hacer el cruce
df_cruce = pd.merge(
    df_arope_clean,
    df_poblacion_arope,
    on=['Año', 'Sexo', 'Edad_AROPE'],
    how='inner'
)

# Calcular personas en AROPE
df_cruce['Personas_AROPE'] = (df_cruce['Valor'] / 100) * df_cruce['Poblacion']

print(f"\n✓ Cruce completado: {len(df_cruce)} registros")
print(f"  Periodo: {df_cruce['Año'].min()} - {df_cruce['Año'].max()}")
print(f"  Sexos: {df_cruce['Sexo'].nunique()}")
print(f"  Grupos edad: {df_cruce['Edad_AROPE'].nunique()}")
print(f"  Esperado: {(df_cruce['Año'].max() - df_cruce['Año'].min() + 1) * df_cruce['Sexo'].nunique() * df_cruce['Edad_AROPE'].nunique()} registros")

print("\n" + "=" * 100)

In [ ]:
# VALIDACIÓN CRÍTICA: Verificar integridad del cruce
print("=" * 100)
print("🔍 VALIDACIÓN DE INTEGRIDAD DE DATOS")
print("=" * 100)

# 1. Verificar que no hay duplicados en df_cruce
print("\n1️⃣ VERIFICACIÓN DE DUPLICADOS:")
duplicados = df_cruce.groupby(['Año', 'Sexo', 'Edad_AROPE']).size()
duplicados_encontrados = duplicados[duplicados > 1]

if len(duplicados_encontrados) > 0:
    print(f"❌ ERROR: Se encontraron {len(duplicados_encontrados)} combinaciones duplicadas:")
    print(duplicados_encontrados)
else:
    print(f"✅ No hay duplicados en df_cruce")

# 2. Verificar que columna 'Indicador' existe y está filtrada
print("\n2️⃣ VERIFICACIÓN DE INDICADOR:")
if 'Indicador' in df_cruce.columns:
    indicadores_unicos = df_cruce['Indicador'].unique()
    print(f"   Indicadores en df_cruce: {indicadores_unicos}")
    if len(indicadores_unicos) == 1 and indicadores_unicos[0] == 'AROPE':
        print(f"   ✅ Solo indicador AROPE (sin componentes)")
    else:
        print(f"   ⚠️ ADVERTENCIA: Hay más de un indicador o no es solo AROPE")
else:
    print(f"   ℹ️ Columna 'Indicador' no presente (posiblemente ya filtrada)")

# 3. Verificar cobertura temporal completa
print("\n3️⃣ VERIFICACIÓN DE COBERTURA TEMPORAL:")
años_esperados = set(range(2008, 2024))
años_reales = set(df_cruce['Año'].unique())
años_faltantes = años_esperados - años_reales

if años_faltantes:
    print(f"   ⚠️ Años faltantes: {sorted(años_faltantes)}")
else:
    print(f"   ✅ Cobertura completa 2008-2023")

# 4. Verificar que todos los grupos de edad están presentes
print("\n4️⃣ VERIFICACIÓN DE GRUPOS DE EDAD:")
grupos_esperados = set(grupos_validos)
grupos_reales = set(df_cruce['Edad_AROPE'].unique())
grupos_faltantes = grupos_esperados - grupos_reales

if grupos_faltantes:
    print(f"   ⚠️ Grupos faltantes: {grupos_faltantes}")
else:
    print(f"   ✅ Todos los grupos de edad presentes: {len(grupos_reales)}")

# 5. Verificar que las tasas AROPE están en rango razonable
print("\n5️⃣ VERIFICACIÓN DE RANGO DE VALORES:")
min_tasa = df_cruce['Valor'].min()
max_tasa = df_cruce['Valor'].max()
print(f"   Rango tasas AROPE: {min_tasa:.1f}% - {max_tasa:.1f}%")

if min_tasa < 0 or max_tasa > 100:
    print(f"   ❌ ERROR: Tasas fuera de rango válido (0-100%)")
elif max_tasa > 60:
    print(f"   ⚠️ ADVERTENCIA: Tasa máxima muy alta ({max_tasa:.1f}%)")
else:
    print(f"   ✅ Tasas en rango razonable")

# 6. Verificar que Personas_AROPE es coherente
print("\n6️⃣ VERIFICACIÓN DE PERSONAS_AROPE:")
personas_negativas = df_cruce[df_cruce['Personas_AROPE'] < 0]
if len(personas_negativas) > 0:
    print(f"   ❌ ERROR: {len(personas_negativas)} registros con personas negativas")
else:
    print(f"   ✅ Todas las personas AROPE >= 0")

# Verificar recálculo
df_cruce['Personas_AROPE_Check'] = (df_cruce['Valor'] / 100) * df_cruce['Poblacion']
diferencias = abs(df_cruce['Personas_AROPE'] - df_cruce['Personas_AROPE_Check'])
if diferencias.max() > 1:
    print(f"   ⚠️ ADVERTENCIA: Diferencias en recálculo (max: {diferencias.max():.1f})")
else:
    print(f"   ✅ Cálculo de Personas_AROPE verificado")
    
df_cruce.drop('Personas_AROPE_Check', axis=1, inplace=True)

print("\n" + "=" * 100)

---

## 3️⃣ Análisis de Vulnerabilidad por EDAD

### 🎯 Objetivos

- **Ranking de vulnerabilidad** por grupo de edad
- **Evolución temporal** (2008-2023) de tasas AROPE
- **Cuantificación** de personas afectadas
- **Índice de concentración** del riesgo
- **Cambios estructurales** por crisis económica y COVID

In [ ]:
# 3.1 - Ranking de Vulnerabilidad por Edad y Cuantificación de Personas Afectadas
print("=" * 100)
print("👥 CUANTIFICACIÓN: Personas en Riesgo por Edad")
print("=" * 100)

# Año más reciente y año inicial para comparaciones
ano_reciente = df_cruce['Año'].max()
ano_inicial = df_cruce['Año'].min()

# DEFINIR ORDEN DE GRUPOS DE EDAD (para consistencia en tablas)
orden_edades = ['Menores de 16 años', '16 a 29 años', '30 a 44 años', '45 a 64 años', '65 y más años']

# Filtrar datos del año más reciente
df_reciente = df_cruce[
    (df_cruce['Año'] == ano_reciente) & 
    (df_cruce['Sexo'] == 'Total')
].sort_values('Valor', ascending=False).copy()

print(f"\n[TABLA INTEGRADA {ano_reciente}] Tasa AROPE + Personas Afectadas por Edad:")
print("-" * 100)
print(f"{'Grupo Edad':<30} {'Tasa AROPE':<12} {'Población':<15} {'Personas AROPE':<18} {'% del Total':<12}")
print("-" * 100)

# Calcular totales
total_pob = df_reciente['Poblacion'].sum()
total_arope = df_reciente['Personas_AROPE'].sum()

for _, row in df_reciente.iterrows():
    edad = row['Edad_AROPE']
    tasa = row['Valor']
    poblacion = row['Poblacion']
    personas = row['Personas_AROPE']
    pct_total = (personas / total_arope) * 100
    
    # Emoji según nivel de vulnerabilidad
    emoji = "🔴" if tasa >= 30 else "🟠" if tasa >= 25 else "🟡" if tasa >= 20 else "🟢"
    
    print(f"{emoji} {edad:<28} {tasa:>5.1f}%       {poblacion:>12,.0f}    {personas:>14,.0f}    {pct_total:>6.1f}%")

print("-" * 100)
print(f"{'TOTAL ESPAÑA':<30} {(total_arope/total_pob*100):>5.1f}%       {total_pob:>12,.0f}    {total_arope:>14,.0f}    100.0%")
print("=" * 100)

# Análisis de magnitud: ¿Cuántas personas representa cada punto porcentual?
print(f"\n💡 MAGNITUD DEL IMPACTO:")
print("-" * 100)
print(f"   • 1 punto porcentual de AROPE ≈ {(total_pob / 100):,.0f} personas")
print(f"   • Total en AROPE ({ano_reciente}): {total_arope:,.0f} personas")
print(f"   • Población total analizada: {total_pob:,.0f} personas")

# Identificar grupos más afectados en términos absolutos
print(f"\n🔴 TOP 3 GRUPOS POR PERSONAS AFECTADAS:")
print("-" * 100)
for i, (_, row) in enumerate(df_reciente.head(3).iterrows(), 1):
    edad = row['Edad_AROPE']
    personas = row['Personas_AROPE']
    tasa = row['Valor']
    pct_del_total = (personas / total_arope) * 100
    
    print(f"   {i}. {edad}")
    print(f"      • {personas:,.0f} personas ({pct_del_total:.1f}% del total en AROPE)")
    print(f"      • Tasa: {tasa:.1f}%")

# Comparación temporal: Cambio absoluto 2008-2023
print(f"\n📊 EVOLUCIÓN TEMPORAL [{ano_inicial} → {ano_reciente}]:")
print("-" * 100)

df_inicial = df_cruce[
    (df_cruce['Año'] == ano_inicial) & 
    (df_cruce['Sexo'] == 'Total')
].set_index('Edad_AROPE')

df_reciente_idx = df_reciente.set_index('Edad_AROPE')

print(f"{'Grupo Edad':<30} {'Tasa Inicial':<15} {'Tasa Final':<15} {'Cambio Tasa':<15} {'Cambio Personas':<20}")
print("-" * 100)

for edad in orden_edades:
    if edad in df_inicial.index and edad in df_reciente_idx.index:
        tasa_inicial = df_inicial.loc[edad, 'Valor']
        tasa_final = df_reciente_idx.loc[edad, 'Valor']
        personas_inicial = df_inicial.loc[edad, 'Personas_AROPE']
        personas_final = df_reciente_idx.loc[edad, 'Personas_AROPE']
        
        cambio_tasa = tasa_final - tasa_inicial
        cambio_personas = personas_final - personas_inicial
        
        emoji_tasa = "📉" if cambio_tasa < 0 else "📈" if cambio_tasa > 0 else "➡️"
        emoji_personas = "📉" if cambio_personas < 0 else "📈"
        
        print(f"{edad:<30} {tasa_inicial:>6.1f}%         {tasa_final:>6.1f}%         {emoji_tasa} {cambio_tasa:>+5.1f}pp       {emoji_personas} {cambio_personas:>+15,.0f}")

# Análisis de impacto combinado
print(f"\n🎯 ANÁLISIS DE IMPACTO COMBINADO:")
print("-" * 100)

for edad in orden_edades:
    if edad in df_inicial.index and edad in df_reciente_idx.index:
        tasa_inicial = df_inicial.loc[edad, 'Valor']
        tasa_final = df_reciente_idx.loc[edad, 'Valor']
        personas_inicial = df_inicial.loc[edad, 'Personas_AROPE']
        personas_final = df_reciente_idx.loc[edad, 'Personas_AROPE']
        
        cambio_tasa = tasa_final - tasa_inicial
        cambio_personas = personas_final - personas_inicial
        
        # Análisis: ¿Mejora/empeora en tasa? ¿Y en personas?
        if abs(cambio_tasa) > 2 or abs(cambio_personas) > 100000:  # Cambios significativos
            print(f"\n   • {edad}:")
            
            if cambio_tasa < -2:
                print(f"     ✅ Tasa MEJORA: {cambio_tasa:.1f}pp")
            elif cambio_tasa > 2:
                print(f"     ⚠️ Tasa EMPEORA: {cambio_tasa:+.1f}pp")
            
            if cambio_personas > 100000:
                print(f"     🔴 AUMENTO de {cambio_personas:,.0f} personas en riesgo")
            elif cambio_personas < -100000:
                print(f"     ✅ REDUCCIÓN de {abs(cambio_personas):,.0f} personas en riesgo")
            
            # Explicar la paradoja si existe
            if cambio_tasa < 0 and cambio_personas > 0:
                print(f"     📌 PARADOJA: Tasa mejora pero más personas afectadas (crecimiento poblacional)")
            elif cambio_tasa > 0 and cambio_personas < 0:
                print(f"     📌 PARADOJA: Tasa empeora pero menos personas afectadas (descenso poblacional)")

print("\n" + "=" * 100)

In [ ]:
# 3.2 - Evolución Temporal por Edad
print("=" * 100)
print("📈 EVOLUCIÓN TEMPORAL DE AROPE POR EDAD (2008-2023)")
print("=" * 100)

# Filtrar Total (ambos sexos)
df_evol_edad = df_cruce[df_cruce['Sexo'] == 'Total'].copy()

# VALIDACIÓN: Verificar duplicados antes de pivot
duplicados_evol = df_evol_edad.groupby(['Año', 'Edad_AROPE']).size()
if (duplicados_evol > 1).any():
    print(f"⚠️ ADVERTENCIA: Hay duplicados en evolución temporal:")
    print(duplicados_evol[duplicados_evol > 1])

# Crear pivot: Años × Grupos Edad (aggfunc='first' explícito para evitar promedios)
pivot_evol = df_evol_edad.pivot_table(
    index='Año', 
    columns='Edad_AROPE', 
    values='Valor',
    aggfunc='first'  # Tomar primer valor si hay duplicados
)

# Ordenar columnas por edad lógica
orden_edades = ['Menores de 16 años', '16 a 29 años', '30 a 44 años', '45 a 64 años', '65 y más años']
pivot_evol = pivot_evol[[col for col in orden_edades if col in pivot_evol.columns]]

# VALIDACIÓN: Verificar años completos
años_esperados = set(range(2008, 2024))
años_reales = set(pivot_evol.index)
if años_faltantes := años_esperados - años_reales:
    print(f"\n⚠️ Años faltantes en evolución: {sorted(años_faltantes)}")

print(f"\n[TABLA] Tasas AROPE por Edad y Año (%):")
print("-" * 100)
print(pivot_evol.to_string())

# Análisis de cambios 2008 → 2023
print(f"\n[CAMBIOS] {pivot_evol.index.min()} → {pivot_evol.index.max()}:")
print("-" * 100)

ano_inicial = pivot_evol.index.min()
ano_final = pivot_evol.index.max()

for edad in pivot_evol.columns:
    valor_inicial = pivot_evol.loc[ano_inicial, edad]
    valor_final = pivot_evol.loc[ano_final, edad]
    cambio = valor_final - valor_inicial
    
    # Calcular máximo y mínimo en el período (volatilidad)
    valor_max = pivot_evol[edad].max()
    valor_min = pivot_evol[edad].min()
    volatilidad = valor_max - valor_min
    
    emoji = "📉" if cambio < 0 else "📈" if cambio > 0 else "➡️"
    tendencia = "MEJORA" if cambio < 0 else "EMPEORA" if cambio > 0 else "ESTABLE"
    
    print(f"{emoji} {edad:<30} {valor_inicial:>5.1f}% → {valor_final:>5.1f}%  ({cambio:+.1f}pp)  [{tendencia}]")
    if volatilidad > 5:
        print(f"   ⚠️  Alta volatilidad: Rango {valor_min:.1f}% - {valor_max:.1f}% (±{volatilidad:.1f}pp)")

print("\n" + "=" * 100)

In [ ]:
# 3.3 - Visualización: Evolución Temporal
print("=" * 100)
print("📊 VISUALIZACIÓN: Evolución de AROPE por Edad")
print("=" * 100)

fig, ax = plt.subplots(figsize=(14, 8))

# Colores diferenciados por grupo
colores = {
    'Menores de 16 años': '#e74c3c',  # Rojo
    '16 a 29 años': '#f39c12',        # Naranja
    '30 a 44 años': '#f1c40f',        # Amarillo
    '45 a 64 años': '#3498db',        # Azul
    '65 y más años': '#2ecc71'        # Verde
}

for edad in pivot_evol.columns:
    color = colores.get(edad, None)
    ax.plot(pivot_evol.index, pivot_evol[edad], 
           marker='o', linewidth=2.5, markersize=6, label=edad, color=color)

# Sombrear períodos de crisis
ax.axvspan(2008, 2013, alpha=0.1, color='red', label='Crisis Financiera')
ax.axvspan(2020, 2021, alpha=0.1, color='orange', label='COVID-19')

ax.set_xlabel('Año', fontsize=12, fontweight='bold')
ax.set_ylabel('Tasa AROPE (%)', fontsize=12, fontweight='bold')
ax.set_title('Evolución de Vulnerabilidad por Edad (2008-2023)', 
            fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='best', fontsize=10, ncol=2)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✓ Gráfico generado")
print("=" * 100)

In [ ]:
# 3.4 - Índice de Concentración del Riesgo
print("=" * 100)
print("⚖️ ÍNDICE DE CONCENTRACIÓN DEL RIESGO")
print("=" * 100)

# Calcular % de personas en AROPE vs % de población
df_concentracion = df_cruce[
    (df_cruce['Año'] == ano_reciente) & 
    (df_cruce['Sexo'] == 'Total')
].copy()

# Calcular porcentajes
total_pob = df_concentracion['Poblacion'].sum()
total_arope = df_concentracion['Personas_AROPE'].sum()

df_concentracion['Pct_Poblacion'] = (df_concentracion['Poblacion'] / total_pob) * 100
df_concentracion['Pct_AROPE'] = (df_concentracion['Personas_AROPE'] / total_arope) * 100

# Índice de concentración: Pct_AROPE / Pct_Poblacion
# > 1.0 = Grupo sobrerepresentado en AROPE
# < 1.0 = Grupo subrepresentado en AROPE
df_concentracion['Indice_Concentracion'] = df_concentracion['Pct_AROPE'] / df_concentracion['Pct_Poblacion']

# Ordenar por índice descendente
df_concentracion = df_concentracion.sort_values('Indice_Concentracion', ascending=False)

# MEJORAR: Calcular umbrales basados en percentiles de los datos
indices = df_concentracion['Indice_Concentracion']
umbral_muy_alto = indices.quantile(0.80)  # Top 20%
umbral_alto = indices.quantile(0.60)      # Top 40%
umbral_bajo = indices.quantile(0.40)      # Bottom 60%

print(f"\n[UMBRALES CALCULADOS] Basados en percentiles:")
print(f"  • MUY ALTA concentración: ≥ {umbral_muy_alto:.2f}x (P80)")
print(f"  • ALTA concentración: ≥ {umbral_alto:.2f}x (P60)")
print(f"  • BAJA concentración: < {umbral_bajo:.2f}x (P40)")

print(f"\n[{ano_reciente}] Índice de Concentración del Riesgo por Edad:")
print("-" * 100)
print(f"{'Edad':<30} {'% Población':<15} {'% AROPE':<15} {'Índice':<15} {'Interpretación'}")
print("-" * 100)

for _, row in df_concentracion.iterrows():
    edad = row['Edad_AROPE']
    pct_pob = row['Pct_Poblacion']
    pct_arope = row['Pct_AROPE']
    indice = row['Indice_Concentracion']
    
    # Interpretación basada en percentiles
    if indice >= umbral_muy_alto:
        interpretacion = "🔴 MUY ALTA concentración"
    elif indice >= umbral_alto:
        interpretacion = "🟠 ALTA concentración"
    elif indice >= umbral_bajo:
        interpretacion = "🟡 Proporcional"
    else:
        interpretacion = "🟢 BAJA concentración"
    
    print(f"{edad:<30} {pct_pob:>6.1f}%          {pct_arope:>6.1f}%          {indice:>6.2f}        {interpretacion}")

print("=" * 100)

# Hallazgos clave
print(f"\n💡 HALLAZGOS CLAVE:")
print("-" * 100)

grupo_max_concentracion = df_concentracion.iloc[0]
grupo_min_concentracion = df_concentracion.iloc[-1]

print(f"\n🔴 MAYOR CONCENTRACIÓN: {grupo_max_concentracion['Edad_AROPE']}")
print(f"   • Representan el {grupo_max_concentracion['Pct_Poblacion']:.1f}% de la población")
print(f"   • Pero son el {grupo_max_concentracion['Pct_AROPE']:.1f}% de las personas en AROPE")
print(f"   • Índice: {grupo_max_concentracion['Indice_Concentracion']:.2f}x")

print(f"\n🟢 MENOR CONCENTRACIÓN: {grupo_min_concentracion['Edad_AROPE']}")
print(f"   • Representan el {grupo_min_concentracion['Pct_Poblacion']:.1f}% de la población")
print(f"   • Solo son el {grupo_min_concentracion['Pct_AROPE']:.1f}% de las personas en AROPE")
print(f"   • Índice: {grupo_min_concentracion['Indice_Concentracion']:.2f}x")

print(f"\n📌 NOTA IMPORTANTE:")
print(f"   El índice de concentración mide PROPORCIÓN, no magnitud absoluta.")
print(f"   Un grupo con baja concentración puede tener muchas personas en AROPE.")
print(f"   Ejemplo: Mayores 65+ (baja concentración) pero 2M+ personas afectadas.")

print("\n" + "=" * 100)

In [ ]:
# 3.5 - Conclusiones: Análisis de Edad
print("=" * 100)
print("📝 CONCLUSIONES: Vulnerabilidad por Edad")
print("=" * 100)

print(f"\n🎯 GRUPOS PRIORITARIOS ({ano_reciente}):\n")

# CORREGIR: Ordenar por TASA AROPE (vulnerabilidad real), NO por índice concentración
# df_concentracion está ordenado por índice → REORDENAR por Valor (tasa)
df_prioridad = df_concentracion.sort_values('Valor', ascending=False).copy()

print("1️⃣ INFANCIA Y JUVENTUD (< 30 años):")
menores16 = df_prioridad[df_prioridad['Edad_AROPE'] == 'Menores de 16 años'].iloc[0]
jovenes = df_prioridad[df_prioridad['Edad_AROPE'] == '16 a 29 años'].iloc[0]

print(f"   • Menores de 16: {menores16['Valor']:.1f}% AROPE ({menores16['Personas_AROPE']:,.0f} personas)")
print(f"     - Concentración: {menores16['Indice_Concentracion']:.2f}x")
print(f"   • Jóvenes 16-29: {jovenes['Valor']:.1f}% AROPE ({jovenes['Personas_AROPE']:,.0f} personas)")
print(f"     - Concentración: {jovenes['Indice_Concentracion']:.2f}x")

print(f"\n2️⃣ ADULTOS (30-64 años):")
adultos30_44 = df_prioridad[df_prioridad['Edad_AROPE'] == '30 a 44 años'].iloc[0]
adultos45_64 = df_prioridad[df_prioridad['Edad_AROPE'] == '45 a 64 años'].iloc[0]

print(f"   • Adultos 30-44: {adultos30_44['Valor']:.1f}% AROPE ({adultos30_44['Personas_AROPE']:,.0f} personas)")
print(f"   • Adultos 45-64: {adultos45_64['Valor']:.1f}% AROPE ({adultos45_64['Personas_AROPE']:,.0f} personas)")

print(f"\n3️⃣ MAYORES (65+ años):")
mayores = df_prioridad[df_prioridad['Edad_AROPE'] == '65 y más años'].iloc[0]

print(f"   • Mayores 65+: {mayores['Valor']:.1f}% AROPE ({mayores['Personas_AROPE']:,.0f} personas)")
print(f"     - Concentración: {mayores['Indice_Concentracion']:.2f}x (BAJA)")

print(f"\n" + "-" * 100)
print(f"\n💡 TENDENCIAS CLAVE ({ano_inicial}-{ano_final}):")
print("-" * 100)

# Analizar tendencias desde pivot_evol
print(f"\n📉 MEJORAS:")
for edad in pivot_evol.columns:
    cambio = pivot_evol.loc[ano_final, edad] - pivot_evol.loc[ano_inicial, edad]
    if cambio < -2:  # Mejora > 2pp
        print(f"   • {edad}: {cambio:.1f}pp")

print(f"\n📈 EMPEORAMIENTOS:")
for edad in pivot_evol.columns:
    cambio = pivot_evol.loc[ano_final, edad] - pivot_evol.loc[ano_inicial, edad]
    if cambio > 2:  # Empeora > 2pp
        print(f"   • {edad}: {cambio:+.1f}pp")

print(f"\n" + "-" * 100)
print(f"\n🎯 RECOMENDACIONES DE POLÍTICA PÚBLICA:")
print("-" * 100)

# CORREGIR: Identificar grupo más vulnerable por TASA, no por índice
# df_prioridad ahora está ordenado correctamente por 'Valor' (tasa AROPE)
mas_vulnerable = df_prioridad.iloc[0]['Edad_AROPE']
tasa_mas_vulnerable = df_prioridad.iloc[0]['Valor']
indice_mas_vulnerable = df_prioridad.iloc[0]['Indice_Concentracion']

print(f"\n1. PRIORIDAD MÁXIMA: {mas_vulnerable}")
print(f"   • Tasa AROPE: {tasa_mas_vulnerable:.1f}% (la más alta)")
print(f"   • Índice concentración: {indice_mas_vulnerable:.2f}x")
print(f"   • Medidas: Prestaciones familiares, ayudas escolares, comedores")

print(f"\n2. PREVENCIÓN: Jóvenes 16-29 años")
print(f"   • Alta vulnerabilidad en transición educación-empleo")
print(f"   • Medidas: Empleo juvenil, formación, vivienda accesible")

print(f"\n3. PROTECCIÓN: Mayores 65+ años")
print(f"   • Baja concentración pero alta población")
print(f"   • Medidas: Pensiones mínimas, dependencia, servicios sociales")

print("\n" + "=" * 100)

---

## 4️⃣ Análisis de Vulnerabilidad por SEXO

### 🎯 Objetivos

- **Brecha de género**: Diferencias en tasas AROPE entre hombres y mujeres
- **Evolución temporal**: Tendencias por sexo (2008-2023)
- **Intersección edad × sexo**: Identificar grupos vulnerables combinados
- **Cuantificación**: Personas afectadas por sexo y grupo de edad

In [ ]:
# 4.1 - Brecha de Género: Análisis Comparativo y Cuantificación
print("=" * 100)
print("⚖️ BRECHA DE GÉNERO EN VULNERABILIDAD")
print("=" * 100)

# DEFINIR ORDEN DE GRUPOS DE EDAD (necesario para celdas posteriores 4.4 y 4.5)
orden_edades = ['Menores de 16 años', '16 a 29 años', '30 a 44 años', '45 a 64 años', '65 y más años']

# Año más reciente
ano_reciente = df_cruce['Año'].max()

# Filtrar por sexo (Hombre y Mujer, excluir Total)
df_sexo = df_cruce[
    (df_cruce['Año'] == ano_reciente) & 
    (df_cruce['Sexo'].isin(['Hombre', 'Mujer']))
].copy()

# Calcular totales por sexo
df_sexo_total = df_sexo.groupby('Sexo', as_index=False).agg({
    'Poblacion': 'sum',
    'Personas_AROPE': 'sum'
})
df_sexo_total['Valor'] = (df_sexo_total['Personas_AROPE'] / df_sexo_total['Poblacion']) * 100

print(f"\n[TABLA INTEGRADA {ano_reciente}] Vulnerabilidad por Sexo:")
print("-" * 100)
print(f"{'Sexo':<20} {'Tasa AROPE':<12} {'Población':<15} {'Personas AROPE':<18} {'% del Total AROPE':<20}")
print("-" * 100)

total_arope_ambos = df_sexo_total['Personas_AROPE'].sum()

for _, row in df_sexo_total.iterrows():
    sexo = row['Sexo']
    tasa = row['Valor']
    poblacion = row['Poblacion']
    personas = row['Personas_AROPE']
    pct_total = (personas / total_arope_ambos) * 100
    
    emoji = "🔴" if tasa >= 27 else "🟠" if tasa >= 25 else "🟡"
    
    print(f"{emoji} {sexo:<18} {tasa:>5.1f}%       {poblacion:>12,.0f}    {personas:>14,.0f}    {pct_total:>6.1f}%")

# Calcular brecha de género
tasa_mujer = df_sexo_total[df_sexo_total['Sexo'] == 'Mujer']['Valor'].values[0]
tasa_hombre = df_sexo_total[df_sexo_total['Sexo'] == 'Hombre']['Valor'].values[0]
brecha_pp = tasa_mujer - tasa_hombre
brecha_relativa = ((tasa_mujer / tasa_hombre) - 1) * 100

personas_mujer = df_sexo_total[df_sexo_total['Sexo'] == 'Mujer']['Personas_AROPE'].values[0]
personas_hombre = df_sexo_total[df_sexo_total['Sexo'] == 'Hombre']['Personas_AROPE'].values[0]
diferencia_personas = personas_mujer - personas_hombre

print("-" * 100)
print(f"\n💡 BRECHA DE GÉNERO:")
print("-" * 100)
print(f"   • Mujeres: {tasa_mujer:.1f}% AROPE ({personas_mujer:,.0f} personas)")
print(f"   • Hombres: {tasa_hombre:.1f}% AROPE ({personas_hombre:,.0f} personas)")
print(f"   • Brecha (puntos porcentuales): {brecha_pp:+.1f}pp")
print(f"   • Brecha (relativa): {brecha_relativa:+.1f}%")
print(f"   • Diferencia en personas: {diferencia_personas:+,.0f}")

if abs(brecha_pp) < 0.5:
    interpretacion = "✅ PARIDAD: Brecha prácticamente inexistente"
elif brecha_pp > 0:
    interpretacion = f"⚠️ BRECHA FAVORABLE A HOMBRES: Mujeres {brecha_pp:.1f}pp más vulnerables"
else:
    interpretacion = f"⚠️ BRECHA FAVORABLE A MUJERES: Hombres {abs(brecha_pp):.1f}pp más vulnerables"

print(f"\n   📊 {interpretacion}")

# Magnitud del impacto
pob_mujer = df_sexo_total[df_sexo_total['Sexo'] == 'Mujer']['Poblacion'].values[0]
pob_hombre = df_sexo_total[df_sexo_total['Sexo'] == 'Hombre']['Poblacion'].values[0]

print(f"\n💡 MAGNITUD DEL IMPACTO:")
print("-" * 100)
print(f"   • 1pp AROPE en mujeres ≈ {(pob_mujer / 100):,.0f} personas")
print(f"   • 1pp AROPE en hombres ≈ {(pob_hombre / 100):,.0f} personas")
print(f"   • Cerrar brecha ({abs(brecha_pp):.1f}pp) = {abs(diferencia_personas):,.0f} personas")

print("\n" + "=" * 100)

In [ ]:
# 4.2 - Evolución Temporal de la Brecha de Género
print("=" * 100)
print("📈 EVOLUCIÓN TEMPORAL: Brecha de Género (2008-2023)")
print("=" * 100)

# Calcular tasas por sexo y año (excluyendo 'Total')
df_evol_sexo = df_cruce[df_cruce['Sexo'].isin(['Hombre', 'Mujer'])].groupby(
    ['Año', 'Sexo'], as_index=False
).agg({
    'Poblacion': 'sum',
    'Personas_AROPE': 'sum'
})
df_evol_sexo['Valor'] = (df_evol_sexo['Personas_AROPE'] / df_evol_sexo['Poblacion']) * 100

# Crear pivot para visualización
pivot_sexo = df_evol_sexo.pivot_table(
    index='Año',
    columns='Sexo',
    values='Valor',
    aggfunc='first'
)

# Calcular brecha por año
pivot_sexo['Brecha_pp'] = pivot_sexo['Mujer'] - pivot_sexo['Hombre']

print(f"\n[TABLA] Tasas AROPE por Sexo y Brecha de Género (%):")
print("-" * 100)
print(f"{'Año':<8} {'Hombres':<12} {'Mujeres':<12} {'Brecha (pp)':<15} {'Tendencia'}")
print("-" * 100)

for año in pivot_sexo.index:
    hombre = pivot_sexo.loc[año, 'Hombre']
    mujer = pivot_sexo.loc[año, 'Mujer']
    brecha = pivot_sexo.loc[año, 'Brecha_pp']
    
    # Emoji según brecha
    if abs(brecha) < 0.5:
        emoji = "⚖️"
        tendencia = "PARIDAD"
    elif brecha > 0:
        emoji = "🔴"
        tendencia = "Pro-Hombre"
    else:
        emoji = "🔵"
        tendencia = "Pro-Mujer"
    
    print(f"{año:<8} {hombre:>6.1f}%       {mujer:>6.1f}%       {emoji} {brecha:>+5.1f}pp      {tendencia}")

# Análisis de cambios
ano_inicial = pivot_sexo.index.min()
ano_final = pivot_sexo.index.max()

print("\n" + "-" * 100)
print(f"\n📊 CAMBIOS [{ano_inicial} → {ano_final}]:")
print("-" * 100)

cambio_hombre = pivot_sexo.loc[ano_final, 'Hombre'] - pivot_sexo.loc[ano_inicial, 'Hombre']
cambio_mujer = pivot_sexo.loc[ano_final, 'Mujer'] - pivot_sexo.loc[ano_inicial, 'Mujer']
cambio_brecha = pivot_sexo.loc[ano_final, 'Brecha_pp'] - pivot_sexo.loc[ano_inicial, 'Brecha_pp']

emoji_h = "📉" if cambio_hombre < 0 else "📈"
emoji_m = "📉" if cambio_mujer < 0 else "📈"

print(f"   {emoji_h} Hombres: {cambio_hombre:+.1f}pp (de {pivot_sexo.loc[ano_inicial, 'Hombre']:.1f}% a {pivot_sexo.loc[ano_final, 'Hombre']:.1f}%)")
print(f"   {emoji_m} Mujeres: {cambio_mujer:+.1f}pp (de {pivot_sexo.loc[ano_inicial, 'Mujer']:.1f}% a {pivot_sexo.loc[ano_final, 'Mujer']:.1f}%)")
print(f"\n   🎯 Brecha: {cambio_brecha:+.1f}pp (de {pivot_sexo.loc[ano_inicial, 'Brecha_pp']:+.1f}pp a {pivot_sexo.loc[ano_final, 'Brecha_pp']:+.1f}pp)")

# Análisis de cambio neto
if abs(cambio_brecha) < 0.3:
    print(f"      ✅ Cambio neto MÍNIMO: La brecha inicio-fin apenas cambió ({cambio_brecha:+.1f}pp)")
elif cambio_brecha > 0:
    print(f"      ⚠️ AUMENTO neto: La brecha se amplió {cambio_brecha:.1f}pp (más desigual)")
else:
    print(f"      ✅ REDUCCIÓN neta: La brecha se redujo {abs(cambio_brecha):.1f}pp (más equitativo)")

# Volatilidad de la brecha
max_brecha = pivot_sexo['Brecha_pp'].max()
min_brecha = pivot_sexo['Brecha_pp'].min()
volatilidad_brecha = max_brecha - min_brecha

print(f"\n   📊 Volatilidad de la brecha: {volatilidad_brecha:.1f}pp")
print(f"      • Máxima brecha: {max_brecha:+.1f}pp (año {pivot_sexo['Brecha_pp'].idxmax()})")
print(f"      • Mínima brecha: {min_brecha:+.1f}pp (año {pivot_sexo['Brecha_pp'].idxmin()})")

# Interpretación de volatilidad
if volatilidad_brecha > 2.5:
    print(f"      🔴 ALTA VOLATILIDAD: La brecha osciló {volatilidad_brecha:.1f}pp durante el período")
    # Detectar inversión de signo
    if (max_brecha > 0 and min_brecha < 0):
        print(f"      ⚠️ INVERSIÓN DE SIGNO: La brecha cambió de dirección durante crisis 2011-2015")
        print(f"         • Hombres más vulnerables en años de crisis (desempleo masculino)")
        print(f"         • Mujeres más vulnerables antes y después de crisis")
elif volatilidad_brecha > 1.5:
    print(f"      🟠 VOLATILIDAD MODERADA: Cambios significativos pero sin cambio de dirección")
else:
    print(f"      ✅ BAJA VOLATILIDAD: Brecha consistente durante el período")

print("\n" + "=" * 100)

In [ ]:
# 4.3 - Visualización: Evolución Brecha de Género
print("=" * 100)
print("📊 VISUALIZACIÓN: Evolución de Brecha de Género")
print("=" * 100)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Tasas por sexo
ax1.plot(pivot_sexo.index, pivot_sexo['Hombre'], 
        marker='o', linewidth=2.5, markersize=6, label='Hombres', color='#3498db')
ax1.plot(pivot_sexo.index, pivot_sexo['Mujer'], 
        marker='o', linewidth=2.5, markersize=6, label='Mujeres', color='#e74c3c')

# Sombrear períodos de crisis
ax1.axvspan(2008, 2013, alpha=0.1, color='red', label='Crisis Financiera')
ax1.axvspan(2020, 2021, alpha=0.1, color='orange', label='COVID-19')

ax1.set_xlabel('Año', fontsize=11, fontweight='bold')
ax1.set_ylabel('Tasa AROPE (%)', fontsize=11, fontweight='bold')
ax1.set_title('Evolución de Vulnerabilidad por Sexo (2008-2023)', 
             fontsize=13, fontweight='bold', pad=15)
ax1.legend(loc='best', fontsize=10)
ax1.grid(True, alpha=0.3)

# Gráfico 2: Brecha de género
ax2.plot(pivot_sexo.index, pivot_sexo['Brecha_pp'], 
        marker='s', linewidth=2.5, markersize=6, color='#9b59b6', label='Brecha (Mujer - Hombre)')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Colorear zona positiva (favorable a hombres) y negativa (favorable a mujeres)
ax2.fill_between(pivot_sexo.index, 0, pivot_sexo['Brecha_pp'], 
                 where=(pivot_sexo['Brecha_pp'] >= 0), alpha=0.3, color='red', 
                 label='Mujeres más vulnerables')
ax2.fill_between(pivot_sexo.index, 0, pivot_sexo['Brecha_pp'], 
                 where=(pivot_sexo['Brecha_pp'] < 0), alpha=0.3, color='blue', 
                 label='Hombres más vulnerables')

ax2.set_xlabel('Año', fontsize=11, fontweight='bold')
ax2.set_ylabel('Brecha (puntos porcentuales)', fontsize=11, fontweight='bold')
ax2.set_title('Evolución de la Brecha de Género en AROPE', 
             fontsize=13, fontweight='bold', pad=15)
ax2.legend(loc='best', fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Gráficos generados")
print("=" * 100)

In [ ]:
# 4.4 - Intersección Edad × Sexo: Heatmap de Vulnerabilidad
print("=" * 100)
print("🔥 INTERSECCIÓN: Vulnerabilidad por Edad y Sexo")
print("=" * 100)

# Año más reciente
ano_reciente = df_cruce['Año'].max()

# Filtrar datos del año más reciente (Hombre y Mujer)
df_interseccion = df_cruce[
    (df_cruce['Año'] == ano_reciente) & 
    (df_cruce['Sexo'].isin(['Hombre', 'Mujer']))
].copy()

# Crear pivot: Edad × Sexo con tasas AROPE
pivot_interseccion = df_interseccion.pivot_table(
    index='Edad_AROPE',
    columns='Sexo',
    values='Valor',
    aggfunc='first'
)

# Definir orden lógico de edad (AUTOSUFICIENTE - no depende de variables externas)
orden_edades_local = ['Menores de 16 años', '16 a 29 años', '30 a 44 años', '45 a 64 años', '65 y más años']

# Reordenar por orden lógico de edad
pivot_interseccion = pivot_interseccion.reindex(orden_edades_local)

# Calcular brecha por grupo de edad
pivot_interseccion['Brecha_pp'] = pivot_interseccion['Mujer'] - pivot_interseccion['Hombre']

# UMBRALES BASADOS EN PERCENTILES (no arbitrarios)
brechas_abs = pivot_interseccion['Brecha_pp'].abs()
umbral_paridad = brechas_abs.quantile(0.25)     # Bottom 25% = paridad
umbral_alta = brechas_abs.quantile(0.75)        # Top 25% = alta brecha

print(f"\n📐 UMBRALES CALCULADOS (basados en distribución real de brechas):")
print(f"   • Paridad:          brecha < {umbral_paridad:.2f}pp (percentil 25)")
print(f"   • Brecha moderada:  brecha entre {umbral_paridad:.2f}pp y {umbral_alta:.2f}pp")
print(f"   • Brecha alta:      brecha ≥ {umbral_alta:.2f}pp (percentil 75)")

print(f"\n[{ano_reciente}] Tasas AROPE por Edad y Sexo:")
print("-" * 100)
print(f"{'Grupo Edad':<30} {'Hombres':<12} {'Mujeres':<12} {'Brecha (pp)':<15} {'Interpretación'}")
print("-" * 100)

# Margen de tolerancia para errores de precisión flotante
TOLERANCIA = 0.01

for edad in pivot_interseccion.index:
    hombre = pivot_interseccion.loc[edad, 'Hombre']
    mujer = pivot_interseccion.loc[edad, 'Mujer']
    brecha = pivot_interseccion.loc[edad, 'Brecha_pp']
    brecha_abs = abs(brecha)
    
    # Interpretación según percentiles (con tolerancia para casos borde)
    if brecha_abs < umbral_paridad - TOLERANCIA:
        emoji = "⚖️"
        interpretacion = "Paridad"
    elif brecha_abs >= umbral_alta - TOLERANCIA:
        # Caso borde: brecha_abs está en el umbral o por encima
        emoji = "🔴"
        if brecha > 0:
            interpretacion = "Alta brecha (Mujeres +)"
        else:
            interpretacion = "Alta brecha (Hombres +)"
    else:
        # Brecha moderada
        if brecha > 0:
            emoji = "🟠"
            interpretacion = "Brecha moderada (Mujeres +)"
        else:
            emoji = "🟦"
            interpretacion = "Brecha moderada (Hombres +)"
    
    print(f"{edad:<30} {hombre:>6.1f}%       {mujer:>6.1f}%       {emoji} {brecha:>+5.1f}pp      {interpretacion}")

print("=" * 100)

# Identificar grupos más vulnerables por sexo
print(f"\n🔴 GRUPOS MÁS VULNERABLES POR SEXO:")
print("-" * 100)

# Para mujeres
vulnerables_mujer = df_interseccion[df_interseccion['Sexo'] == 'Mujer'].nlargest(3, 'Valor')
print(f"\n   MUJERES:")
for i, (_, row) in enumerate(vulnerables_mujer.iterrows(), 1):
    edad = row['Edad_AROPE']
    tasa = row['Valor']
    personas = row['Personas_AROPE']
    print(f"   {i}. {edad}: {tasa:.1f}% ({personas:,.0f} personas)")

# Para hombres
vulnerables_hombre = df_interseccion[df_interseccion['Sexo'] == 'Hombre'].nlargest(3, 'Valor')
print(f"\n   HOMBRES:")
for i, (_, row) in enumerate(vulnerables_hombre.iterrows(), 1):
    edad = row['Edad_AROPE']
    tasa = row['Valor']
    personas = row['Personas_AROPE']
    print(f"   {i}. {edad}: {tasa:.1f}% ({personas:,.0f} personas)")

# Identificar mayor brecha
mayor_brecha_idx = pivot_interseccion['Brecha_pp'].abs().idxmax()
mayor_brecha_valor = pivot_interseccion.loc[mayor_brecha_idx, 'Brecha_pp']

print(f"\n🎯 MAYOR BRECHA DE GÉNERO:")
print(f"   • Grupo: {mayor_brecha_idx}")
print(f"   • Brecha: {mayor_brecha_valor:+.1f}pp")
if mayor_brecha_valor > 0:
    print(f"   • Mujeres más vulnerables en este grupo")
else:
    print(f"   • Hombres más vulnerables en este grupo")

In [ ]:
# 4.5 - Visualización: Heatmap Edad × Sexo
print("=" * 100)
print("📊 VISUALIZACIÓN: Heatmap de Vulnerabilidad por Edad y Sexo")
print("=" * 100)

fig, ax = plt.subplots(figsize=(12, 7))

# Preparar datos para heatmap (solo tasas Hombre y Mujer)
heatmap_data = pivot_interseccion[['Hombre', 'Mujer']].T

# Crear heatmap
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn_r', 
           cbar_kws={'label': 'Tasa AROPE (%)'}, 
           linewidths=0.5, linecolor='gray',
           vmin=0, vmax=40, ax=ax)

ax.set_xlabel('Grupo de Edad', fontsize=12, fontweight='bold')
ax.set_ylabel('Sexo', fontsize=12, fontweight='bold')
ax.set_title(f'Vulnerabilidad por Edad y Sexo ({ano_reciente})', 
            fontsize=14, fontweight='bold', pad=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("✓ Heatmap generado")
print("=" * 100)

In [ ]:
# 4.6 - Conclusiones: Análisis por Sexo
print("=" * 100)
print("📝 CONCLUSIONES: Vulnerabilidad por Sexo")
print("=" * 100)

print(f"\n🎯 HALLAZGOS CLAVE ({ano_reciente}):\n")

print("1️⃣ BRECHA DE GÉNERO GLOBAL:")
print(f"   • Brecha actual: {brecha_pp:+.1f}pp")
if abs(brecha_pp) < 0.5:
    print(f"   • ✅ PARIDAD prácticamente alcanzada a nivel nacional")
elif brecha_pp > 0:
    print(f"   • ⚠️ Mujeres {brecha_pp:.1f}pp más vulnerables que hombres")
    print(f"   • Equivale a {abs(diferencia_personas):,.0f} personas más en riesgo")
else:
    print(f"   • ⚠️ Hombres {abs(brecha_pp):.1f}pp más vulnerables que mujeres")
    print(f"   • Equivale a {abs(diferencia_personas):,.0f} personas más en riesgo")

print(f"\n2️⃣ EVOLUCIÓN TEMPORAL:")
print(f"   • Cambio neto ({ano_inicial}→{ano_final}): {cambio_brecha:+.1f}pp")
print(f"   • Volatilidad periodo: {volatilidad_brecha:.1f}pp (rango: {min_brecha:+.1f} a {max_brecha:+.1f}pp)")

# INTERPRETACIÓN CORREGIDA: Distinguir cambio neto vs volatilidad
if abs(cambio_brecha) < 0.5 and volatilidad_brecha < 1.5:
    print(f"   • ✅ ESTABLE: Cambio neto y volatilidad bajos")
elif abs(cambio_brecha) < 0.5 and volatilidad_brecha >= 2.5:
    print(f"   • 🟡 CAMBIO NETO BAJO pero ALTA VOLATILIDAD")
    print(f"   • ⚠️ La brecha osciló significativamente durante el período")
    # Detectar inversión de signo
    if (max_brecha > 0.5 and min_brecha < -0.5):
        print(f"   • 🚨 INVERSIÓN DE SIGNO DETECTADA:")
        print(f"      - Crisis 2012-2015: Hombres temporalmente más vulnerables")
        print(f"      - Causa: Desempleo masculino (construcción/industria)")
        print(f"      - Post-crisis: Brecha retorna a patrón tradicional (mujeres +)")
elif cambio_brecha > 0.5:
    print(f"   • ⚠️ Brecha AUMENTÓ (más desigual)")
else:
    print(f"   • ✅ Brecha REDUJO (más equitativo)")

print(f"\n3️⃣ INTERSECCIONALIDAD (Edad × Sexo):")
print(f"   • Mayor brecha: {mayor_brecha_idx} ({mayor_brecha_valor:+.1f}pp)")
if mayor_brecha_valor > 0:
    print(f"   • Mujeres especialmente vulnerables en este grupo")
else:
    print(f"   • Hombres especialmente vulnerables en este grupo")

# Contar grupos usando percentiles CON TOLERANCIA (consistente con celda 4.4)
TOLERANCIA = 0.01
grupos_paridad = (pivot_interseccion['Brecha_pp'].abs() < umbral_paridad - TOLERANCIA).sum()
grupos_brecha_alta = (pivot_interseccion['Brecha_pp'].abs() >= umbral_alta - TOLERANCIA).sum()
grupos_moderada = len(pivot_interseccion) - grupos_paridad - grupos_brecha_alta

print(f"\n   • Grupos en paridad (< p25={umbral_paridad:.2f}pp): {grupos_paridad}/{len(pivot_interseccion)}")
print(f"   • Grupos brecha moderada (p25-p75): {grupos_moderada}/{len(pivot_interseccion)}")
print(f"   • Grupos brecha alta (≥ p75={umbral_alta:.2f}pp): {grupos_brecha_alta}/{len(pivot_interseccion)}")

print(f"\n" + "-" * 100)

print(f"\n🎯 RECOMENDACIONES DE POLÍTICA PÚBLICA:")
print("-" * 100)

print(f"\n1. PRIORIDAD: Reducir brecha de género")
print(f"   • Focalizar en mujeres del grupo: {mayor_brecha_idx}")
print(f"   • Medidas específicas por género en grupos con alta brecha")

print(f"\n2. ENFOQUE INTERSECCIONAL:")
print(f"   • No basta analizar sexo aisladamente")
print(f"   • Edad + Sexo revelan vulnerabilidades específicas")
print(f"   • Diseñar políticas considerando ambas dimensiones")

print(f"\n3. MONITOREO CONTINUO:")
print(f"   • Volatilidad detectada: {volatilidad_brecha:.1f}pp")
if volatilidad_brecha > 2.5:
    print(f"   • 🚨 ALTA VOLATILIDAD requiere seguimiento trimestral")
    print(f"   • Implementar alertas ante cambios bruscos (crisis, reformas)")
else:
    print(f"   • Seguimiento anual suficiente")

print("\n" + "=" * 100)

---

## 5️⃣ Análisis de Vulnerabilidad por TIPO DE HOGAR

### 🎯 Objetivos

- **Ranking de vulnerabilidad**: Identificar tipos de hogar más afectados
- **Evolución temporal**: Tendencias por tipo de hogar (2013-2023)
- **Limitaciones**: Solo tasas AROPE disponibles (sin datos poblacionales para cuantificar personas)
- **Nota metodológica**: Período limitado 2013-2023 (no disponible 2008-2012)

In [ ]:
# 5.1 - Ranking de Vulnerabilidad por Tipo de Hogar
print("=" * 100)
print("🏠 VULNERABILIDAD POR TIPO DE HOGAR")
print("=" * 100)

# Verificar disponibilidad de datos
ano_inicio_hogar = df_arope_hogar['Año'].min()
ano_fin_hogar = df_arope_hogar['Año'].max()

print(f"\n⚠️ LIMITACIÓN DE DATOS:")
print(f"   • Período disponible: {ano_inicio_hogar}-{ano_fin_hogar} ({ano_fin_hogar - ano_inicio_hogar + 1} años)")
print(f"   • Período completo deseable: 2008-2023 (16 años)")
print(f"   • Años faltantes: 2008-{ano_inicio_hogar-1}")
print(f"   • METODOLOGÍA: Solo análisis de TASAS (sin cuantificación de personas)")
print(f"      Razón: No disponemos de datos poblacionales por tipo de hogar")

# Filtrar solo AROPE total (no componentes) Y EXCLUIR "No consta" Y "Total"
df_hogar = df_arope_hogar[
    (df_arope_hogar['Indicador'] == 'AROPE') & 
    (df_arope_hogar['Tipo_Hogar'] != 'No consta') &
    (df_arope_hogar['Tipo_Hogar'] != 'Total')
].copy()

print(f"\n📌 NOTA METODOLÓGICA:")
print(f"   • Se excluyen categorías 'No consta' (artefacto) y 'Total' (agregado)")
print(f"   • Razón: 'No consta' no es tipo de hogar real, 'Total' contamina dispersión")

# Año más reciente
ano_reciente_hogar = df_hogar['Año'].max()

# Obtener tasa Total para comparación (antes de filtrar)
tasa_total = df_arope_hogar[
    (df_arope_hogar['Indicador'] == 'AROPE') & 
    (df_arope_hogar['Tipo_Hogar'] == 'Total') &
    (df_arope_hogar['Año'] == ano_reciente_hogar)
]['Valor'].values[0]

# Filtrar año más reciente y ordenar por tasa
df_hogar_reciente = df_hogar[df_hogar['Año'] == ano_reciente_hogar].sort_values('Valor', ascending=False)

print(f"\n📊 REFERENCIA: Tasa AROPE Total {ano_reciente_hogar} = {tasa_total:.1f}%")
print(f"\n[RANKING {ano_reciente_hogar}] Tasas AROPE por Tipo de Hogar:")
print("-" * 100)
print(f"{'Tipo de Hogar':<60} {'Tasa AROPE':<15}")
print("-" * 100)

for idx, row in df_hogar_reciente.iterrows():
    tipo_hogar = row['Tipo_Hogar']
    tasa = row['Valor']
    
    # Emoji según nivel de vulnerabilidad
    if tasa >= 40:
        emoji = "🔴"
    elif tasa >= 30:
        emoji = "🟠"
    elif tasa >= 20:
        emoji = "🟡"
    else:
        emoji = "🟢"
    
    print(f"{emoji} {tipo_hogar:<58} {tasa:>6.1f}%")

print("=" * 100)

# Identificar grupos más vulnerables
grupo_mas_vulnerable = df_hogar_reciente.iloc[0]
grupo_menos_vulnerable = df_hogar_reciente.iloc[-1]

print(f"\n🔴 TIPO DE HOGAR MÁS VULNERABLE:")
print(f"   • {grupo_mas_vulnerable['Tipo_Hogar']}")
print(f"   • Tasa AROPE: {grupo_mas_vulnerable['Valor']:.1f}%")

print(f"\n🟢 TIPO DE HOGAR MENOS VULNERABLE:")
print(f"   • {grupo_menos_vulnerable['Tipo_Hogar']}")
print(f"   • Tasa AROPE: {grupo_menos_vulnerable['Valor']:.1f}%")

print(f"\n📊 DISPERSIÓN:")
brecha_hogar = grupo_mas_vulnerable['Valor'] - grupo_menos_vulnerable['Valor']
print(f"   • Diferencia entre extremos: {brecha_hogar:.1f}pp")
print(f"   • Factor de riesgo vs menos vulnerable: {(grupo_mas_vulnerable['Valor'] / grupo_menos_vulnerable['Valor']):.1f}x")
print(f"   • Factor de riesgo vs Total: {(grupo_mas_vulnerable['Valor'] / tasa_total):.2f}x")

print("\n" + "=" * 100)


In [ ]:
# 5.2 - Evolución Temporal por Tipo de Hogar
print("=" * 100)
print("📈 EVOLUCIÓN TEMPORAL: Vulnerabilidad por Tipo de Hogar")
print("=" * 100)

# Calcular cambios entre año inicial y final
ano_inicial_hogar = df_hogar['Año'].min()
ano_final_hogar = df_hogar['Año'].max()

print(f"\n⚠️ CONTEXTO TEMPORAL:")
print(f"   • Año inicial ({ano_inicial_hogar}): FUE AÑO PICO DE CRISIS FINANCIERA")
print(f"   • Comparaciones inicio-fin pueden reflejar recuperación post-crisis")
print(f"   • Recomendación: Interpretar cambios como DESCRIPTIVOS, no como tendencias")

df_hogar_inicial = df_hogar[df_hogar['Año'] == ano_inicial_hogar].set_index('Tipo_Hogar')
df_hogar_final = df_hogar[df_hogar['Año'] == ano_final_hogar].set_index('Tipo_Hogar')

print(f"\n[CAMBIOS {ano_inicial_hogar} → {ano_final_hogar}]:")
print("-" * 100)
print(f"{'Tipo de Hogar':<60} {'Tasa Inicial':<15} {'Tasa Final':<15} {'Cambio':<15}")
print("-" * 100)

cambios = []

for tipo_hogar in df_hogar_final.index:
    if tipo_hogar in df_hogar_inicial.index:
        tasa_inicial = df_hogar_inicial.loc[tipo_hogar, 'Valor']
        tasa_final = df_hogar_final.loc[tipo_hogar, 'Valor']
        cambio = tasa_final - tasa_inicial
        
        cambios.append({
            'Tipo_Hogar': tipo_hogar,
            'Cambio': cambio
        })
        
        # Emoji según dirección del cambio
        emoji = "📉" if cambio < -2 else "📈" if cambio > 2 else "➡️"
        
        print(f"{tipo_hogar:<60} {tasa_inicial:>6.1f}%         {tasa_final:>6.1f}%         {emoji} {cambio:>+5.1f}pp")

print("=" * 100)

# Análisis de cambios con justificación de umbral
df_cambios = pd.DataFrame(cambios)

# Calcular percentiles para justificar umbral
cambios_abs = df_cambios['Cambio'].abs()
p50_cambio = cambios_abs.median()

print(f"\n📊 JUSTIFICACIÓN UMBRAL:")
print("-" * 100)
print(f"   • Mediana de cambios absolutos (P50): {p50_cambio:.1f}pp")
print(f"   • Umbral usado: 2.00pp")
print(f"   • Criterio: Umbral CONSERVADOR (< P50) para captar más cambios")
print(f"   • Rationale: Con solo {len(df_cambios)} tipos de hogar, umbral bajo evita perder información")

reducciones_significativas = df_cambios[df_cambios['Cambio'] < -2].sort_values('Cambio')
aumentos_significativos = df_cambios[df_cambios['Cambio'] > 2].sort_values('Cambio', ascending=False)

if len(reducciones_significativas) > 0:
    print(f"\n📉 REDUCCIONES SIGNIFICATIVAS (> 2pp desde pico crisis):")
    print("-" * 100)
    for idx, row in reducciones_significativas.iterrows():
        print(f"   • {row['Tipo_Hogar']}: {row['Cambio']:.1f}pp")

if len(aumentos_significativos) > 0:
    print(f"\n📈 AUMENTOS SIGNIFICATIVOS (> 2pp desde pico crisis):")
    print("-" * 100)
    for idx, row in aumentos_significativos.iterrows():
        print(f"   • {row['Tipo_Hogar']}: {row['Cambio']:+.1f}pp")

if len(reducciones_significativas) == 0 and len(aumentos_significativos) == 0:
    print(f"\n➡️ ESTABILIDAD RELATIVA:")
    print(f"   • No hay cambios significativos (> 2pp) en ningún tipo de hogar")
    print(f"   • La estructura de vulnerabilidad por hogar se mantiene similar")

print(f"\n📌 INTERPRETACIÓN:")
print(f"   • Cambios muestran RECUPERACIÓN POST-CRISIS (2013 fue pico)")
print(f"   • NO implican necesariamente tendencia sostenida")
print(f"   • Análisis longitudinal completo requeriría datos desde 2008")

print("\n" + "=" * 100)


In [ ]:
# VERIFICACIÓN CRÍTICA: ¿Son 2pp y 33.5pp umbrales válidos?
print("=" * 100)
print("🔍 AUDITORÍA: Verificación de Umbrales y Métricas de Dispersión")
print("=" * 100)

# 1. VERIFICAR UMBRAL 2pp - ¿Es arbitrario o basado en datos?
print("\n1️⃣ VERIFICACIÓN UMBRAL '2pp' PARA CAMBIOS SIGNIFICATIVOS:")
print("-" * 100)

cambios_abs = df_cambios['Cambio'].abs()
p25_cambio = cambios_abs.quantile(0.25)
p75_cambio = cambios_abs.quantile(0.75)
mediana_cambio = cambios_abs.median()

print(f"   • Percentil 25 de cambios (P25): {p25_cambio:.2f}pp")
print(f"   • Mediana de cambios (P50):      {mediana_cambio:.2f}pp")
print(f"   • Percentil 75 de cambios (P75): {p75_cambio:.2f}pp")
print(f"   • Umbral usado actualmente:      2.00pp")

if p75_cambio < 2.0:
    print(f"\n   ⚠️  PROBLEMA: Umbral 2pp está SOBRE P75 ({p75_cambio:.2f}pp)")
    print(f"      → Clasificas como 'no significativo' cambios que están en top 25%")
    print(f"      → SUGERENCIA: Usar P75 = {p75_cambio:.2f}pp como umbral")
elif p75_cambio > 2.0:
    print(f"\n   ✅ RAZONABLE: Umbral 2pp está entre P50 y P75")
    print(f"      → Clasificación conservadora de 'significativo'")
else:
    print(f"\n   ✅ COINCIDE: Umbral 2pp = P75 exacto")

# 2. VERIFICAR DISPERSIÓN - ¿Brecha extremos o métrica robusta?
print(f"\n2️⃣ VERIFICACIÓN MÉTRICA DE DISPERSIÓN:")
print("-" * 100)

tasas_2023 = df_hogar_reciente['Valor'].values
desv_std = np.std(tasas_2023, ddof=1)  # Desviación estándar muestral
coef_var = (desv_std / np.mean(tasas_2023)) * 100
rango_iq = np.percentile(tasas_2023, 75) - np.percentile(tasas_2023, 25)

print(f"   • Brecha extremos (usado actual):  {brecha_hogar:.1f}pp")
print(f"   • Desviación estándar (robusta):   {desv_std:.1f}pp")
print(f"   • Rango intercuartil P75-P25:      {rango_iq:.1f}pp")
print(f"   • Coeficiente de variación:        {coef_var:.1f}%")

print(f"\n   📊 INTERPRETACIÓN:")
if desv_std < brecha_hogar / 2:
    print(f"      • Desv.std ({desv_std:.1f}pp) << Brecha extremos ({brecha_hogar:.1f}pp)")
    print(f"      • PROBLEMA: Extremos son OUTLIERS, no representan dispersión típica")
    print(f"      • SUGERENCIA: Reportar desv.std como medida principal")
else:
    print(f"      • Desv.std coherente con brecha extremos")
    
# 3. VERIFICAR si 16.8% (menos vulnerable) es outlier
print(f"\n3️⃣ VERIFICACIÓN: ¿Es 16.8% un OUTLIER?")
print("-" * 100)

q1 = np.percentile(tasas_2023, 25)
q3 = np.percentile(tasas_2023, 75)
iqr = q3 - q1
limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

print(f"   • Q1 (P25):              {q1:.1f}%")
print(f"   • Q3 (P75):              {q3:.1f}%")
print(f"   • IQR (Q3-Q1):           {iqr:.1f}pp")
print(f"   • Límite outlier bajo:   {limite_inferior:.1f}%")
print(f"   • Límite outlier alto:   {limite_superior:.1f}%")

if grupo_menos_vulnerable['Valor'] < limite_inferior:
    print(f"\n   🚨 OUTLIER CONFIRMADO: {grupo_menos_vulnerable['Valor']:.1f}% < {limite_inferior:.1f}%")
    print(f"      • '{grupo_menos_vulnerable['Tipo_Hogar']}' es estadísticamente atípico")
    print(f"      • Factor 3.0x exagerado por comparar con outlier")
elif grupo_mas_vulnerable['Valor'] > limite_superior:
    print(f"\n   🚨 OUTLIER CONFIRMADO: {grupo_mas_vulnerable['Valor']:.1f}% > {limite_superior:.1f}%")
    print(f"      • '{grupo_mas_vulnerable['Tipo_Hogar']}' es estadísticamente atípico")
else:
    print(f"\n   ✅ NO OUTLIERS: Extremos dentro de rango esperado")

# 4. ¿Cuántos tipos de hogar tenemos realmente?
print(f"\n4️⃣ INFORMACIÓN CONTEXTUAL:")
print("-" * 100)
print(f"   • Tipos de hogar en análisis (tras filtros): {len(df_hogar_reciente)}")
print(f"   • TOP 5 representa:                          {(5/len(df_hogar_reciente))*100:.0f}% del total")

if len(df_hogar_reciente) <= 6:
    print(f"\n   ⚠️  LIMITACIÓN: Con solo {len(df_hogar_reciente)} categorías, TOP 5 es casi todo")
    print(f"      → No hay suficiente granularidad para análisis detallado")

print("\n" + "=" * 100)


In [ ]:
# 5.3 - Visualización: Evolución por Tipo de Hogar (5 de 6 tipos)
print("=" * 100)
print("📊 VISUALIZACIÓN: Evolución Tipos de Hogar Más Vulnerables")
print("=" * 100)

# Identificar 5 tipos de hogar más vulnerables en el año más reciente
# (Ya excluye "No consta" y "Total" por filtro en celda 5.1)
top5_hogares = df_hogar_reciente.head(5)['Tipo_Hogar'].tolist()

print(f"\n📌 Tipos de hogar mostrados (5 de 6 totales):")
for i, tipo in enumerate(top5_hogares, 1):
    print(f"   {i}. {tipo}")

print(f"\n⚠️  NOTA: Con solo 6 tipos tras filtros, este gráfico muestra 83% de categorías")

# Filtrar datos solo para esos 5 tipos
df_hogar_top5 = df_hogar[df_hogar['Tipo_Hogar'].isin(top5_hogares)]

# Crear figura
fig, ax = plt.subplots(figsize=(14, 8))

# Paleta de colores
colores = ['#e74c3c', '#e67e22', '#f39c12', '#16a085', '#3498db']

for i, tipo_hogar in enumerate(top5_hogares):
    df_tipo = df_hogar_top5[df_hogar_top5['Tipo_Hogar'] == tipo_hogar].sort_values('Año')
    ax.plot(df_tipo['Año'], df_tipo['Valor'], 
           marker='o', linewidth=2.5, markersize=7, 
           label=tipo_hogar, color=colores[i])

# Sombrear crisis COVID
ax.axvspan(2020, 2021, alpha=0.1, color='orange', label='COVID-19')

ax.set_xlabel('Año', fontsize=12, fontweight='bold')
ax.set_ylabel('Tasa AROPE (%)', fontsize=12, fontweight='bold')
ax.set_title(f'Evolución 5 de 6 Tipos de Hogar (2013-2023)', 
            fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='best', fontsize=9, framealpha=0.9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Gráfico generado (excluye 'No consta' y 'Total')")
print("=" * 100)


In [ ]:
# 5.4 - Conclusiones: Análisis por Tipo de Hogar
print("=" * 100)
print("📝 CONCLUSIONES: Vulnerabilidad por Tipo de Hogar")
print("=" * 100)

print(f"\n🎯 HALLAZGOS CLAVE ({ano_reciente_hogar}):\n")

print("1️⃣ TIPOLOGÍA MÁS VULNERABLE:")
print(f"   • {grupo_mas_vulnerable['Tipo_Hogar']}")
print(f"   • Tasa AROPE: {grupo_mas_vulnerable['Valor']:.1f}%")
print(f"   • Factor de riesgo vs menos vulnerable: {(grupo_mas_vulnerable['Valor'] / grupo_menos_vulnerable['Valor']):.1f}x")
print(f"   • Factor de riesgo vs Total: {(grupo_mas_vulnerable['Valor'] / tasa_total):.2f}x")

# ✅ AÑADIDO: Nota sobre outlier estadístico
print(f"\n   🚨 ADVERTENCIA ESTADÍSTICA:")
print(f"      • {grupo_mas_vulnerable['Tipo_Hogar']} es OUTLIER estadístico")
print(f"      • Tasa {grupo_mas_vulnerable['Valor']:.1f}% > límite superior P75+1.5×IQR")
print(f"      • Factor 3.0x amplificado por comparar outlier con valor normal")

print(f"\n2️⃣ DISPERSIÓN POR TIPO DE HOGAR:")
# ✅ AÑADIDO: Desviación estándar como métrica principal
tasas_hogar_2023 = df_hogar_reciente['Valor'].values
desv_std_hogar = np.std(tasas_hogar_2023, ddof=1)
coef_var_hogar = (desv_std_hogar / np.mean(tasas_hogar_2023)) * 100

print(f"   • Desviación estándar (métrica robusta): {desv_std_hogar:.1f}pp")
print(f"   • Diferencia entre extremos:             {brecha_hogar:.1f}pp")
print(f"   • Coeficiente de variación:              {coef_var_hogar:.1f}%")

# ✅ MODIFICADO: Criterio basado en coef.variación, no brecha extremos
if coef_var_hogar > 50:
    print(f"   • 🔴 ALTÍSIMA HETEROGENEIDAD: Tipología de hogar está fuertemente asociada con vulnerabilidad")
elif coef_var_hogar > 30:
    print(f"   • 🟠 ALTA HETEROGENEIDAD: Tipología importante pero no determinante")
else:
    print(f"   • 🟡 HETEROGENEIDAD MODERADA")

print(f"\n   📌 INTERPRETACIÓN:")
print(f"      • Desv.std {desv_std_hogar:.1f}pp representa dispersión típica")
print(f"      • Brecha extremos {brecha_hogar:.1f}pp influida por outlier monoparental")

print(f"\n3️⃣ EVOLUCIÓN TEMPORAL ({ano_inicial_hogar}→{ano_final_hogar}):")
if len(reducciones_significativas) > 0:
    print(f"   • ✅ Reducciones desde pico crisis (2013) en {len(reducciones_significativas)} tipo(s) de hogar")
if len(aumentos_significativos) > 0:
    print(f"   • ⚠️ Aumentos significativos en {len(aumentos_significativos)} tipo(s) de hogar")
if len(reducciones_significativas) == 0 and len(aumentos_significativos) == 0:
    print(f"   • ➡️ Estabilidad relativa: Sin cambios significativos desde pico crisis")

print(f"\n   📌 NOTA: Análisis excluye 'No consta' (artefacto) y 'Total' (agregado)")

print(f"\n" + "-" * 100)

print(f"\n⚠️ LIMITACIONES METODOLÓGICAS:")
print("-" * 100)
print(f"\n1. PERÍODO LIMITADO:")
print(f"   • Solo disponible {ano_inicio_hogar}-{ano_final_hogar} ({ano_final_hogar - ano_inicio_hogar + 1} años)")
print(f"   • Faltan datos 2008-{ano_inicio_hogar-1} (crisis financiera)")
print(f"   • Imposible analizar impacto de crisis 2008-2013 por hogar")

print(f"\n2. SESGO TEMPORAL:")
print(f"   • Año inicial (2013) fue PICO DE CRISIS FINANCIERA")
print(f"   • Comparaciones 2013→2023 muestran RECUPERACIÓN POST-CRISIS")
print(f"   • NO representan necesariamente tendencias sostenidas a largo plazo")

print(f"\n3. SIN CUANTIFICACIÓN DE PERSONAS:")
print(f"   • Solo disponibles TASAS AROPE")
print(f"   • No hay datos de población por tipo de hogar en INE")
print(f"   • Imposible calcular: personas afectadas, cambios absolutos")
print(f"   • RECOMENDACIÓN: Solicitar a INE microdatos EPA para cruzar")

print(f"\n4. GRANULARIDAD LIMITADA:")
print(f"   • Solo 6 tipos de hogar tras filtros (excluidos 'No consta' y 'Total')")
print(f"   • Categorías amplias ocultan heterogeneidad interna")
print(f"   • Ejemplo: '1 adulto + niños' no distingue 1 vs 3+ niños")

print(f"\n" + "-" * 100)

print(f"\n🎯 RECOMENDACIONES DE POLÍTICA PÚBLICA:")
print("-" * 100)

# ✅ MODIFICADO: Añadir caveat sobre falta de datos de volumen
print(f"\n⚠️  IMPORTANTE: Recomendaciones basadas SOLO EN TASAS, no en volumen de personas")
print(f"   → Requiere datos de población por tipo de hogar para priorización real")

print(f"\n1. PRIORIDAD ALTA (sujeta a verificación de volumen):")
print(f"   • Intervención en {grupo_mas_vulnerable['Tipo_Hogar']}")
print(f"   • Tasa 50.3% AROPE (outlier estadístico)")
print(f"   • Diseñar políticas específicas para esta tipología")
print(f"   • Abordar causas estructurales: ingresos, custodia, conciliación")

print(f"\n2. ENFOQUE DIFERENCIADO:")
print(f"   • La heterogeneidad ({desv_std_hogar:.1f}pp desv.std) justifica políticas diferenciadas")
print(f"   • No aplicar medidas genéricas \"para todas las familias\"")
print(f"   • Focalizar recursos en hogares con niños dependientes (mayor vulnerabilidad)")

print(f"\n3. MEJORAR DATOS (CRÍTICO):")
print(f"   • Solicitar a INE: datos poblacionales por tipo de hogar")
print(f"   • Extender serie temporal a 2008 (si disponible)")
print(f"   • Microdatos EPA para cruzar hogar × edad × sexo × empleo")
print(f"   • SIN ESTOS DATOS: Imposible priorizar por impacto real (personas afectadas)")

print("\n" + "=" * 100)


---

## 6️⃣ Análisis por SITUACIÓN LABORAL

**Objetivo:** Analizar vulnerabilidad según situación laboral (ocupados, parados, jubilados, otros inactivos)

**Limitaciones conocidas:**
- Período: 2014-2023 (10 años) - Más corto que otras dimensiones
- Sin datos población por situación laboral → Solo análisis de tasas
- Año inicial 2014: Post-crisis, sesgo temporal


In [ ]:
# 6.1 - Ranking de Vulnerabilidad por Situación Laboral
print("=" * 100)
print("💼 VULNERABILIDAD POR SITUACIÓN LABORAL")
print("=" * 100)

# Estructura: Sexo, Situacion_Laboral, Territorio, Año, AROPE
# Filtrar solo España y promediar Hombres+Mujeres (no hay 'Total')
df_laboral_esp = df_arope_laboral[df_arope_laboral['Territorio'] == 'España'].copy()

# Agrupar por Año y Situacion_Laboral promediando Hombres y Mujeres
df_laboral = df_laboral_esp.groupby(['Año', 'Situacion_Laboral'], as_index=False)['AROPE'].mean()
df_laboral = df_laboral.rename(columns={'AROPE': 'Valor'})

print(f"\n📌 NOTA METODOLÓGICA:")
print(f"   • Tasas AROPE: Promedio de Hombres y Mujeres (no hay 'Total' en tabla)")
print(f"   • Territorio: Solo España")

# Verificar disponibilidad de datos
ano_inicio_laboral = df_laboral['Año'].min()
ano_fin_laboral = df_laboral['Año'].max()

print(f"\n⚠️ LIMITACIÓN DE DATOS:")
print(f"   • Período disponible: {ano_inicio_laboral}-{ano_fin_laboral} ({ano_fin_laboral - ano_inicio_laboral + 1} años)")
print(f"   • Período completo deseable: 2008-2023 (16 años)")
print(f"   • Años faltantes: 2008-{ano_inicio_laboral-1} (incluye inicio crisis 2008-2013)")
print(f"   • METODOLOGÍA: Solo análisis de TASAS (sin cuantificación de personas)")
print(f"      Razón: No disponemos de datos poblacionales por situación laboral")

# Año más reciente
ano_reciente_laboral = df_laboral['Año'].max()

# Filtrar año más reciente y ordenar por tasa
df_laboral_reciente = df_laboral[df_laboral['Año'] == ano_reciente_laboral].sort_values('Valor', ascending=False)

print(f"\n[RANKING {ano_reciente_laboral}] Tasas AROPE por Situación Laboral:")
print("-" * 100)
print(f"{'Situación Laboral':<60} {'Tasa AROPE':<15}")
print("-" * 100)

for idx, row in df_laboral_reciente.iterrows():
    situacion = row['Situacion_Laboral']
    tasa = row['Valor']
    
    # Emoji según nivel de vulnerabilidad
    if tasa >= 40:
        emoji = "🔴"
    elif tasa >= 30:
        emoji = "🟠"
    elif tasa >= 20:
        emoji = "🟡"
    else:
        emoji = "🟢"
    
    print(f"{emoji} {situacion:<58} {tasa:>6.1f}%")

print("=" * 100)

# Identificar situaciones más vulnerables
situacion_mas_vulnerable = df_laboral_reciente.iloc[0]
situacion_menos_vulnerable = df_laboral_reciente.iloc[-1]

print(f"\n🔴 SITUACIÓN LABORAL MÁS VULNERABLE:")
print(f"   • {situacion_mas_vulnerable['Situacion_Laboral']}")
print(f"   • Tasa AROPE: {situacion_mas_vulnerable['Valor']:.1f}%")

print(f"\n🟢 SITUACIÓN LABORAL MENOS VULNERABLE:")
print(f"   • {situacion_menos_vulnerable['Situacion_Laboral']}")
print(f"   • Tasa AROPE: {situacion_menos_vulnerable['Valor']:.1f}%")

print(f"\n📊 DISPERSIÓN:")
brecha_laboral = situacion_mas_vulnerable['Valor'] - situacion_menos_vulnerable['Valor']
print(f"   • Diferencia entre extremos: {brecha_laboral:.1f}pp")
print(f"   • Factor de riesgo vs menos vulnerable: {(situacion_mas_vulnerable['Valor'] / situacion_menos_vulnerable['Valor']):.1f}x")

# Calcular métricas robustas de dispersión
tasas_laboral_2023 = df_laboral_reciente['Valor'].values
desv_std_laboral = np.std(tasas_laboral_2023, ddof=1)
coef_var_laboral = (desv_std_laboral / np.mean(tasas_laboral_2023)) * 100

print(f"\n📈 MÉTRICAS ROBUSTAS DE DISPERSIÓN:")
print(f"   • Desviación estándar: {desv_std_laboral:.1f}pp")
print(f"   • Coeficiente de variación: {coef_var_laboral:.1f}%")

# Verificar outliers
q1_lab = np.percentile(tasas_laboral_2023, 25)
q3_lab = np.percentile(tasas_laboral_2023, 75)
iqr_lab = q3_lab - q1_lab
limite_superior_lab = q3_lab + 1.5 * iqr_lab

if situacion_mas_vulnerable['Valor'] > limite_superior_lab:
    print(f"\n⚠️  ADVERTENCIA: '{situacion_mas_vulnerable['Situacion_Laboral']}' es outlier estadístico")
    print(f"   • Tasa {situacion_mas_vulnerable['Valor']:.1f}% > límite superior {limite_superior_lab:.1f}%")

print("\n" + "=" * 100)


In [ ]:
# 6.2 - Evolución Temporal por Situación Laboral
print("=" * 100)
print("📈 EVOLUCIÓN TEMPORAL: Vulnerabilidad por Situación Laboral")
print("=" * 100)

# Calcular cambios entre año inicial y final
ano_inicial_laboral = df_laboral['Año'].min()
ano_final_laboral = df_laboral['Año'].max()

print(f"\n⚠️ CONTEXTO TEMPORAL:")
print(f"   • Año inicial ({ano_inicial_laboral}): POST-CRISIS (pico fue 2013)")
print(f"   • Faltan años clave: 2008-2013 (inicio y pico de crisis)")
print(f"   • Recomendación: Interpretar cambios como DESCRIPTIVOS, no como tendencias completas")

df_laboral_inicial = df_laboral[df_laboral['Año'] == ano_inicial_laboral].set_index('Situacion_Laboral')
df_laboral_final = df_laboral[df_laboral['Año'] == ano_final_laboral].set_index('Situacion_Laboral')

print(f"\n[CAMBIOS {ano_inicial_laboral} → {ano_final_laboral}]:")
print("-" * 100)
print(f"{'Situación Laboral':<60} {'Tasa Inicial':<15} {'Tasa Final':<15} {'Cambio':<15}")
print("-" * 100)

cambios_laboral = []

for situacion in df_laboral_final.index:
    if situacion in df_laboral_inicial.index:
        tasa_inicial = df_laboral_inicial.loc[situacion, 'Valor']
        tasa_final = df_laboral_final.loc[situacion, 'Valor']
        cambio = tasa_final - tasa_inicial
        
        cambios_laboral.append({
            'Situacion_Laboral': situacion,
            'Cambio': cambio
        })
        
        # Calcular umbral dinámico (después de recopilar todos los cambios)
        # Por ahora uso umbral conservador 2pp
        emoji = "📉" if cambio < -2 else "📈" if cambio > 2 else "➡️"
        
        print(f"{situacion:<60} {tasa_inicial:>6.1f}%         {tasa_final:>6.1f}%         {emoji} {cambio:>+5.1f}pp")

print("=" * 100)

# Análisis de cambios con umbral justificado
df_cambios_laboral = pd.DataFrame(cambios_laboral)

# Calcular percentiles para justificar umbral
cambios_abs_lab = df_cambios_laboral['Cambio'].abs()
p50_cambio_lab = cambios_abs_lab.median()

print(f"\n📊 JUSTIFICACIÓN UMBRAL:")
print("-" * 100)
print(f"   • Mediana de cambios absolutos (P50): {p50_cambio_lab:.1f}pp")
print(f"   • Umbral usado: 2.00pp")
if p50_cambio_lab > 2:
    print(f"   • Criterio: Umbral CONSERVADOR (< P50) para captar más cambios")
else:
    print(f"   • Criterio: Umbral cercano/sobre P50, clasificación estándar")
print(f"   • Rationale: Con solo {len(df_cambios_laboral)} situaciones, umbral bajo evita perder información")

reducciones_laboral = df_cambios_laboral[df_cambios_laboral['Cambio'] < -2].sort_values('Cambio')
aumentos_laboral = df_cambios_laboral[df_cambios_laboral['Cambio'] > 2].sort_values('Cambio', ascending=False)

if len(reducciones_laboral) > 0:
    print(f"\n📉 REDUCCIONES SIGNIFICATIVAS (> 2pp desde {ano_inicial_laboral}):")
    print("-" * 100)
    for idx, row in reducciones_laboral.iterrows():
        print(f"   • {row['Situacion_Laboral']}: {row['Cambio']:.1f}pp")

if len(aumentos_laboral) > 0:
    print(f"\n📈 AUMENTOS SIGNIFICATIVOS (> 2pp desde {ano_inicial_laboral}):")
    print("-" * 100)
    for idx, row in aumentos_laboral.iterrows():
        print(f"   • {row['Situacion_Laboral']}: {row['Cambio']:+.1f}pp")

if len(reducciones_laboral) == 0 and len(aumentos_laboral) == 0:
    print(f"\n➡️ ESTABILIDAD RELATIVA:")
    print(f"   • No hay cambios significativos (> 2pp) en ninguna situación")
    print(f"   • La estructura de vulnerabilidad laboral se mantiene similar")

print(f"\n📌 INTERPRETACIÓN:")
print(f"   • Año inicial {ano_inicial_laboral} ya es POST-CRISIS (pico 2013)")
print(f"   • Cambios desde {ano_inicial_laboral} muestran evolución en fase de recuperación")
print(f"   • Análisis longitudinal completo requeriría datos desde 2008")

print("\n" + "=" * 100)


In [ ]:
# 6.3 - Visualización: Evolución por Situación Laboral
print("=" * 100)
print("📊 VISUALIZACIÓN: Evolución por Situación Laboral")
print("=" * 100)

# Todas las situaciones (son pocas, mostrarlas todas)
situaciones_mostrar = df_laboral_reciente['Situacion_Laboral'].tolist()

print(f"\n📌 Situaciones laborales mostradas ({len(situaciones_mostrar)} totales):")
for i, sit in enumerate(situaciones_mostrar, 1):
    print(f"   {i}. {sit}")

# Filtrar datos para visualización
df_laboral_viz = df_laboral[df_laboral['Situacion_Laboral'].isin(situaciones_mostrar)]

# Crear figura
fig, ax = plt.subplots(figsize=(14, 8))

# Paleta de colores (adaptada según número de situaciones)
if len(situaciones_mostrar) == 4:
    colores = ['#e74c3c', '#e67e22', '#16a085', '#3498db']
elif len(situaciones_mostrar) == 3:
    colores = ['#e74c3c', '#f39c12', '#3498db']
else:
    colores = plt.cm.Set2(np.linspace(0, 1, len(situaciones_mostrar)))

for i, situacion in enumerate(situaciones_mostrar):
    df_sit = df_laboral_viz[df_laboral_viz['Situacion_Laboral'] == situacion].sort_values('Año')
    ax.plot(df_sit['Año'], df_sit['Valor'], 
           marker='o', linewidth=2.5, markersize=7, 
           label=situacion, color=colores[i])

# Sombrear crisis COVID
ax.axvspan(2020, 2021, alpha=0.1, color='orange', label='COVID-19')

ax.set_xlabel('Año', fontsize=12, fontweight='bold')
ax.set_ylabel('Tasa AROPE (%)', fontsize=12, fontweight='bold')
ax.set_title(f'Evolución Vulnerabilidad por Situación Laboral ({ano_inicial_laboral}-{ano_final_laboral})', 
            fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='best', fontsize=10, framealpha=0.9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Gráfico generado")
print("=" * 100)


In [ ]:
# 6.4 - Conclusiones: Análisis por Situación Laboral
print("=" * 100)
print("📝 CONCLUSIONES: Vulnerabilidad por Situación Laboral")
print("=" * 100)

print(f"\n🎯 HALLAZGOS CLAVE ({ano_reciente_laboral}):\n")

print("1️⃣ SITUACIÓN LABORAL MÁS VULNERABLE:")
print(f"   • {situacion_mas_vulnerable['Situacion_Laboral']}")
print(f"   • Tasa AROPE: {situacion_mas_vulnerable['Valor']:.1f}%")
print(f"   • Factor de riesgo vs menos vulnerable: {(situacion_mas_vulnerable['Valor'] / situacion_menos_vulnerable['Valor']):.1f}x")

# Verificar si hay outlier
if situacion_mas_vulnerable['Valor'] > limite_superior_lab:
    print(f"\n   🚨 ADVERTENCIA ESTADÍSTICA:")
    print(f"      • {situacion_mas_vulnerable['Situacion_Laboral']} es OUTLIER estadístico")
    print(f"      • Tasa {situacion_mas_vulnerable['Valor']:.1f}% > límite superior P75+1.5×IQR")

print(f"\n2️⃣ DISPERSIÓN POR SITUACIÓN LABORAL:")
print(f"   • Desviación estándar (métrica robusta): {desv_std_laboral:.1f}pp")
print(f"   • Diferencia entre extremos:             {brecha_laboral:.1f}pp")
print(f"   • Coeficiente de variación:              {coef_var_laboral:.1f}%")

# Interpretación basada en coef.variación
if coef_var_laboral > 50:
    print(f"   • 🔴 ALTÍSIMA HETEROGENEIDAD: Situación laboral está fuertemente asociada con vulnerabilidad")
elif coef_var_laboral > 30:
    print(f"   • 🟠 ALTA HETEROGENEIDAD: Situación laboral importante pero no determinante")
else:
    print(f"   • 🟡 HETEROGENEIDAD MODERADA")

print(f"\n3️⃣ EVOLUCIÓN TEMPORAL ({ano_inicial_laboral}→{ano_final_laboral}):")
if len(reducciones_laboral) > 0:
    print(f"   • ✅ Reducciones desde {ano_inicial_laboral} en {len(reducciones_laboral)} situación(es)")
if len(aumentos_laboral) > 0:
    print(f"   • ⚠️ Aumentos significativos en {len(aumentos_laboral)} situación(es)")
if len(reducciones_laboral) == 0 and len(aumentos_laboral) == 0:
    print(f"   • ➡️ Estabilidad relativa: Sin cambios significativos")

print(f"\n   📌 NOTA: Tasas promediadas de Hombres y Mujeres (no hay 'Total' en BD)")

print(f"\n" + "-" * 100)

print(f"\n⚠️ LIMITACIONES METODOLÓGICAS:")
print("-" * 100)
print(f"\n1. PERÍODO MÁS LIMITADO:")
print(f"   • Solo disponible {ano_inicial_laboral}-{ano_final_laboral} ({ano_final_laboral - ano_inicial_laboral + 1} años)")
print(f"   • Faltan datos 2008-{ano_inicial_laboral-1} (CRÍTICO: incluye inicio y pico crisis)")
print(f"   • Año inicial {ano_inicial_laboral} es POST-CRISIS, NO captura impacto 2008-2013")
print(f"   • Imposible analizar cómo crisis afectó inicialmente a parados vs ocupados")

print(f"\n2. SESGO TEMPORAL MÁS SEVERO:")
print(f"   • Perdemos años 2008-2013 donde tasa de paro pasó 8%→26%")
print(f"   • Comparaciones desde {ano_inicial_laboral} muestran solo fase de recuperación")
print(f"   • NO representan ciclo completo (crisis→recuperación)")

print(f"\n3. SIN CUANTIFICACIÓN DE PERSONAS:")
print(f"   • Solo disponibles TASAS AROPE")
print(f"   • No hay datos de población por situación laboral en esta tabla")
print(f"   • Imposible calcular: personas afectadas, cambios absolutos")
print(f"   • RECOMENDACIÓN: Cruzar con EPA para obtener volúmenes")

print(f"\n4. GRANULARIDAD LIMITADA:")
print(f"   • Solo {len(df_laboral_reciente)} situaciones laborales")
print(f"   • No distingue: tipo de contrato, jornada, sector, antigüedad")
print(f"   • 'Ocupados' agrupa desde precarios a estables")
print(f"   • Oculta fenómeno 'working poor' (pobreza laboral)")

print(f"\n5. PROMEDIO HOMBRES-MUJERES:")
print(f"   • BD no tiene 'Total', se promedian tasas H/M")
print(f"   • Puede ocultar diferencias de género en situación laboral")
print(f"   • Idealmente debería ponderarse por población de cada sexo")

print(f"\n" + "-" * 100)

print(f"\n🎯 RECOMENDACIONES DE POLÍTICA PÚBLICA:")
print("-" * 100)

print(f"\n⚠️  IMPORTANTE: Recomendaciones basadas SOLO EN TASAS, no en volumen de personas")
print(f"   → Requiere datos EPA para cuantificar impacto real")

print(f"\n1. PRIORIDAD ALTA (sujeta a verificación de volumen):")
print(f"   • Intervención en {situacion_mas_vulnerable['Situacion_Laboral']}")
print(f"   • Tasa {situacion_mas_vulnerable['Valor']:.1f}% AROPE")
print(f"   • Políticas de inserción laboral y protección social")

print(f"\n2. MONITOREAR 'WORKING POOR' (POBREZA LABORAL):")
print(f"   • Situación 'Ocupados' puede tener tasas AROPE significativas")
print(f"   • Indica precariedad: bajos salarios, jornadas reducidas, temporalidad")
print(f"   • Políticas: salario mínimo, calidad empleo, negociación colectiva")

print(f"\n3. ANÁLISIS DIFERENCIADO POR TIPO DE EMPLEO:")
print(f"   • 'Ocupados' es categoría demasiado agregada")
print(f"   • Necesario desagregar: temporal/indefinido, tiempo completo/parcial")
print(f"   • Cruzar con datos EPA para análisis detallado")

print(f"\n4. AMPLIAR SERIE HISTÓRICA (CRÍTICO):")
print(f"   • Solicitar a INE: extender serie a 2008-2013")
print(f"   • Permitiría analizar impacto completo de crisis 2008")
print(f"   • Esencial para evaluar políticas laborales del período")

print("\n" + "=" * 100)


---

## 7️⃣ SÍNTESIS Y RECOMENDACIONES

**Objetivo:** Integrar hallazgos de las 4 dimensiones analizadas y priorizar grupos vulnerables

**Alcance:**
- Ranking de dimensiones por poder explicativo (coeficiente de variación)
- Identificación de grupos críticos por dimensión
- Limitaciones para análisis multidimensional (requiere microdatos)
- Recomendaciones de política pública basadas en evidencia

**Nota metodológica:** No se realizan cruces multidimensionales (edad×hogar, hogar×laboral) por falta de datos desagregados en tablas INE

In [ ]:
# 7.1 - Ranking de Dimensiones por Heterogeneidad
print("=" * 100)
print("📊 RANKING DE DIMENSIONES POR PODER EXPLICATIVO")
print("=" * 100)

print("\n🎯 MÉTRICA PRINCIPAL: Coeficiente de Variación (CV)")
print("-" * 100)
print("   • CV = (Desviación Estándar / Media) × 100")
print("   • Mide heterogeneidad RELATIVA (independiente de escala)")
print("   • Permite comparar dimensiones con diferentes magnitudes")
print("")
print("   Interpretación:")
print("   • CV > 50%:  ALTÍSIMA heterogeneidad → Dimensión MUY estratificante")
print("   • CV 30-50%: ALTA heterogeneidad → Dimensión importante")
print("   • CV 15-30%: MODERADA heterogeneidad → Dimensión relevante")
print("   • CV < 15%:  BAJA heterogeneidad → Dimensión poco diferenciadora")

# Usar métricas ya calculadas en secciones anteriores
# EDAD: Calcular CV aproximado (no tenemos todas las variables, estimamos)
# Basándonos en la sección 3, sabemos que tasas van desde ~14% (65+) a ~29% (menores 16)
# Estimación conservadora: CV ~35% (dispersión moderada-alta)
coef_var_edad_estimado = 35.0
desv_std_edad_estimado = 7.0  # Estimado

# SEXO: Brecha de género es pequeña (~1.6pp), CV bajo
# Con tasas ~25% y ~26.6%, desv.std ~1.1pp, CV ~4.4%
tasa_media_sexo = 25.8  # Promedio aproximado H/M
brecha_genero_abs = 1.6  # De la sección 4
desv_std_sexo_calc = brecha_genero_abs / np.sqrt(2)  # ~1.13pp
coef_var_sexo_calc = (desv_std_sexo_calc / tasa_media_sexo) * 100  # ~4.4%

# Recopilar métricas de todas las secciones
dimensiones_ranking = pd.DataFrame({
    'Dimensión': ['SITUACIÓN LABORAL', 'TIPO DE HOGAR', 'EDAD', 'SEXO'],
    'Coef_Variacion': [coef_var_laboral, coef_var_hogar, coef_var_edad_estimado, coef_var_sexo_calc],
    'Desv_Std_pp': [desv_std_laboral, desv_std_hogar, desv_std_edad_estimado, desv_std_sexo_calc],
    'Periodo': ['2014-2023', '2013-2023', '2008-2023', '2008-2023'],
    'N_Categorias': [4, 6, 6, 2],
    'Grupo_Mas_Vulnerable': ['Parados (62.1%)', 'Monoparentales (50.3%)', 'Menores 16 años (28.9%)', 'Mujeres (+1.6pp)']
})

# Ordenar por coeficiente de variación (descendente)
dimensiones_ranking = dimensiones_ranking.sort_values('Coef_Variacion', ascending=False).reset_index(drop=True)

print("\n" + "=" * 100)
print("[RANKING] Dimensiones por Poder Explicativo de Vulnerabilidad")
print("=" * 100)
print(f"{'#':<5} {'Dimensión':<25} {'Coef.Var':<12} {'Desv.Std':<12} {'Categorías':<12} {'Grupo Más Vulnerable':<35}")
print("-" * 100)

for idx, row in dimensiones_ranking.iterrows():
    rank = idx + 1
    dimension = row['Dimensión']
    cv = row['Coef_Variacion']
    desv = row['Desv_Std_pp']
    n_cat = row['N_Categorias']
    grupo = row['Grupo_Mas_Vulnerable']
    
    # Emoji según nivel de heterogeneidad
    if cv > 50:
        emoji = "🔴"
        nivel = "ALTÍSIMA"
    elif cv > 30:
        emoji = "🟠"
        nivel = "ALTA"
    elif cv > 15:
        emoji = "🟡"
        nivel = "MODERADA"
    else:
        emoji = "🟢"
        nivel = "BAJA"
    
    print(f"{emoji} {rank:<3} {dimension:<25} {cv:>6.1f}%      {desv:>6.1f}pp      {n_cat:<12} {grupo:<35}")

print("=" * 100)

print("\n📈 INTERPRETACIÓN DEL RANKING:")
print("-" * 100)

# Análisis del líder
dim_lider = dimensiones_ranking.iloc[0]
print(f"\n1️⃣ DIMENSIÓN MÁS ESTRATIFICANTE: {dim_lider['Dimensión']}")
print(f"   • Coeficiente de variación: {dim_lider['Coef_Variacion']:.1f}% (ALTÍSIMA heterogeneidad)")
print(f"   • Interpretación: {dim_lider['Dimensión'].capitalize()} es el factor que MAYOR desigualdad")
print(f"     genera en riesgo de exclusión social en España")
print(f"   • Grupo crítico: {dim_lider['Grupo_Mas_Vulnerable']}")

# Comparación 1º vs 2º
if len(dimensiones_ranking) >= 2:
    dim_2 = dimensiones_ranking.iloc[1]
    diferencia_cv = dim_lider['Coef_Variacion'] - dim_2['Coef_Variacion']
    print(f"\n2️⃣ COMPARACIÓN CON 2ª DIMENSIÓN ({dim_2['Dimensión']}):")
    print(f"   • {dim_lider['Dimensión']} tiene {diferencia_cv:.1f}pp más de CV que {dim_2['Dimensión']}")
    print(f"   • Factor de superioridad: {(dim_lider['Coef_Variacion'] / dim_2['Coef_Variacion']):.2f}x")
    print(f"   • Conclusión: {dim_lider['Dimensión'].capitalize()} estratifica {diferencia_cv:.0f}% más que {dim_2['Dimensión'].lower()}")

# Análisis de SEXO (última posición esperada)
dim_sexo = dimensiones_ranking[dimensiones_ranking['Dimensión'] == 'SEXO']
if not dim_sexo.empty:
    cv_sexo = dim_sexo.iloc[0]['Coef_Variacion']
    print(f"\n3️⃣ DIMENSIÓN SEXO (Brecha de Género):")
    print(f"   • Coeficiente de variación: {cv_sexo:.1f}% (BAJA heterogeneidad)")
    print(f"   • Interpretación: Aunque existe brecha de género (+1.6pp a favor mujeres 2023),")
    print(f"     es MUCHO MENOR que diferencias por empleo, hogar o edad")
    print(f"   • Implicación: Sexo solo no es buen predictor; importa MÁS en intersección")
    print(f"     (ej: mujeres monoparentales, mujeres paradas)")

print("\n" + "=" * 100)

print("\n⚠️ LIMITACIONES DE ESTA COMPARACIÓN:")
print("-" * 100)
print("\n1. PERÍODOS TEMPORALES DIFERENTES:")
print("   • EDAD y SEXO: 2008-2023 (16 años)")
print("   • HOGAR: 2013-2023 (11 años)")
print("   • LABORAL: 2014-2023 (10 años)")
print("   • Implicación: No son estrictamente comparables (diferentes contextos históricos)")

print("\n2. NÚMERO DE CATEGORÍAS DIFERENTE:")
print("   • SEXO: 2 categorías (Hombres/Mujeres)")
print("   • LABORAL: 4 categorías (Ocupados/Parados/Jubilados/Inactivos)")
print("   • HOGAR: 6 categorías (tras filtros)")
print("   • EDAD: 6 categorías (grupos etarios)")
print("   • Coef.variación mitiga este sesgo (métrica relativa), pero no lo elimina")

print("\n3. SIN DATOS DE INTERSECCIONALIDAD:")
print("   • Ranking basado en análisis UNIVARIANTE (una dimensión a la vez)")
print("   • Realidad: Vulnerabilidad es MULTIDIMENSIONAL (ej: mujer + monoparental + parada)")
print("   • Solución: Requiere microdatos EPA para análisis multinivel")

print("\n4. AGREGACIÓN TERRITORIAL:")
print("   • Todas las tasas son PROMEDIO NACIONAL (España)")
print("   • Oculta heterogeneidad REGIONAL (Andalucía vs País Vasco, etc.)")
print("   • Análisis regional disponible en notebook 05_analisis_geografico_ccaa")

print("\n5. NOTA SOBRE MÉTRICAS EDAD:")
print("   • CV EDAD (35.0%) es ESTIMADO basado en dispersión observada en sección 3")
print("   • Rango tasas 2023: ~13.9% (65+) a ~28.9% (menores 16)")
print("   • Estimación conservadora, orden de magnitud correcto")

print("\n" + "=" * 100)

In [ ]:
# 7.2 - Grupos Críticos Identificados por Dimensión
print("=" * 100)
print("🎯 GRUPOS CRÍTICOS IDENTIFICADOS")
print("=" * 100)

print("\n📋 RESUMEN DE VULNERABILIDADES POR DIMENSIÓN (Año más reciente):\n")

# Usar valores ya calculados en secciones anteriores
# EDAD - De la sección 3
tasa_menores_16 = 28.9  # Valor de 2023
tasa_65_mas = 13.9      # Valor de 2023

# SEXO - De la sección 4
tasa_mujeres = 26.6     # Valor de 2023
tasa_hombres = 25.0     # Valor de 2023
brecha_genero = 1.6     # Valor de 2023

# HOGAR - De la sección 5
tasa_monoparental = 50.3
tasa_2adultos_sin_ninos = 16.8

# LABORAL - De la sección 6
tasa_parados = 62.1
tasa_ocupados = 15.6

# Crear tabla resumen de grupos críticos
grupos_criticos = []

grupos_criticos.append({
    'Dimensión': 'EDAD',
    'Grupo_Vulnerable': 'Menores de 16 años',
    'Tasa_AROPE': tasa_menores_16,
    'Factor_vs_Menos_Vulnerable': tasa_menores_16 / tasa_65_mas,
    'Outlier_Estadistico': 'No',
    'Comentario': 'Infancia como grupo de máximo riesgo estructural'
})

grupos_criticos.append({
    'Dimensión': 'SEXO',
    'Grupo_Vulnerable': 'Mujeres',
    'Tasa_AROPE': tasa_mujeres,
    'Factor_vs_Menos_Vulnerable': tasa_mujeres / tasa_hombres,
    'Outlier_Estadistico': 'N/A',
    'Comentario': f'Brecha de género: +{brecha_genero:.1f}pp (moderada)'
})

grupos_criticos.append({
    'Dimensión': 'HOGAR',
    'Grupo_Vulnerable': 'Monoparentales (1 adulto + niños)',
    'Tasa_AROPE': tasa_monoparental,
    'Factor_vs_Menos_Vulnerable': tasa_monoparental / tasa_2adultos_sin_ninos,
    'Outlier_Estadistico': 'Sí',
    'Comentario': 'Outlier estadístico (>P75+1.5×IQR), vulnerabilidad extrema'
})

grupos_criticos.append({
    'Dimensión': 'LABORAL',
    'Grupo_Vulnerable': 'Parados',
    'Tasa_AROPE': tasa_parados,
    'Factor_vs_Menos_Vulnerable': tasa_parados / tasa_ocupados,
    'Outlier_Estadistico': 'No',
    'Comentario': 'Máxima vulnerabilidad pero no outlier (límite >100%)'
})

df_criticos = pd.DataFrame(grupos_criticos)

print("-" * 100)
print(f"{'Dimensión':<12} {'Grupo Vulnerable':<35} {'Tasa AROPE':<12} {'Factor':<10} {'Outlier':<10}")
print("-" * 100)

for idx, row in df_criticos.iterrows():
    dimension = row['Dimensión']
    grupo = row['Grupo_Vulnerable']
    tasa = row['Tasa_AROPE']
    factor = row['Factor_vs_Menos_Vulnerable']
    outlier = row['Outlier_Estadistico']
    
    # Emoji según tasa
    if tasa >= 50:
        emoji = "🔴🔴"
    elif tasa >= 30:
        emoji = "🔴"
    elif tasa >= 20:
        emoji = "🟠"
    else:
        emoji = "🟡"
    
    print(f"{emoji} {dimension:<10} {grupo:<35} {tasa:>6.1f}%       {factor:>5.1f}x     {outlier:<10}")
    print(f"   └─ {row['Comentario']}")
    print()

print("=" * 100)

print("\n🔍 ANÁLISIS COMPARATIVO DE VULNERABILIDADES:")
print("-" * 100)

# Identificar grupo con MAYOR tasa AROPE
grupo_max = df_criticos.loc[df_criticos['Tasa_AROPE'].idxmax()]

print(f"\n1️⃣ GRUPO CON MÁXIMA VULNERABILIDAD:")
print(f"   • {grupo_max['Grupo_Vulnerable']} ({grupo_max['Dimensión']})")
print(f"   • Tasa AROPE: {grupo_max['Tasa_AROPE']:.1f}%")
print(f"   • Riesgo {grupo_max['Factor_vs_Menos_Vulnerable']:.1f}x superior a grupo menos vulnerable de su dimensión")

if grupo_max['Outlier_Estadistico'] == 'Sí':
    print(f"   • ⚠️ OUTLIER ESTADÍSTICO: Vulnerabilidad extrema no explicada solo por heterogeneidad normal")

print(f"\n2️⃣ CONVERGENCIA EN INFANCIA:")
print(f"   • Menores de 16 años: {tasa_menores_16:.1f}% AROPE")
print(f"   • Grupos con alta presencia infantil también vulnerables:")
print(f"     - Monoparentales: {tasa_monoparental:.1f}% (mayoría con niños dependientes)")
print(f"   • Hipótesis: Presencia de menores es FACTOR DE RIESGO transversal")
print(f"   • Limitación: No podemos verificar sin microdatos (edad × hogar)")

print(f"\n3️⃣ EMPLEO COMO PROTECCIÓN:")
print(f"   • Ocupados: {tasa_ocupados:.1f}% AROPE (menor vulnerabilidad en dimensión laboral)")
print(f"   • Jubilados: 15.9% AROPE (pensiones como protección)")
print(f"   • Contraste: Parados {tasa_parados:.1f}% vs Ocupados {tasa_ocupados:.1f}% → Factor 4.0x")
print(f"   • Interpretación: Empleo es FACTOR PROTECTOR crítico (pero no suficiente)")
print(f"   • Evidencia de 'working poor': {tasa_ocupados:.1f}% ocupados en AROPE → Precariedad laboral")

print(f"\n4️⃣ BRECHA DE GÉNERO (Moderada en agregado):")
print(f"   • Brecha nacional: +{brecha_genero:.1f}pp a favor de mujeres")
print(f"   • Menor que otras desigualdades (edad, empleo, hogar)")
print(f"   • Hipótesis: Brecha se AMPLIFICA en intersecciones:")
print(f"     - Mujeres monoparentales (doble vulnerabilidad)")
print(f"     - Mujeres paradas (desempleo + brecha)")
print(f"   • Limitación: No verificable sin cruces multidimensionales")

print("\n" + "=" * 100)

print("\n💡 PERFILES DE ALTO RIESGO HIPOTÉTICOS (NO VERIFICADOS):")
print("-" * 100)
print("\n⚠️ ADVERTENCIA: Los siguientes perfiles son INFERENCIAS basadas en tasas agregadas.")
print("   NO hay datos de cruces reales en tablas INE disponibles.")
print("   Se requieren microdatos EPA para confirmar.")

print("\n🔴 PERFIL 1: Menor en hogar monoparental con adulto parado")
print("   • Vulnerabilidades combinadas:")
print(f"     - Menor de 16 años: {tasa_menores_16:.1f}% AROPE (grupo edad)")
print(f"     - Monoparental: {tasa_monoparental:.1f}% AROPE (tipo hogar)")
print(f"     - Parado: {tasa_parados:.1f}% AROPE (situación laboral)")
print("   • Riesgo estimado: EXTREMO (probablemente >70% AROPE)")
print("   • ⚠️ NO VERIFICADO: Supone independencia de factores (falacia ecológica)")

print("\n🔴 PERFIL 2: Mujer monoparental parada")
print("   • Vulnerabilidades combinadas:")
print(f"     - Mujer: +{brecha_genero:.1f}pp brecha género")
print(f"     - Monoparental: {tasa_monoparental:.1f}% AROPE")
print(f"     - Parada: {tasa_parados:.1f}% AROPE")
print("   • Riesgo estimado: MUY ALTO")
print("   • ⚠️ NO VERIFICADO: Requiere cruce hogar × empleo × sexo")

print("\n🟠 PERFIL 3: Joven (16-29) en hogar monoparental")
print("   • Vulnerabilidades combinadas:")
print("     - Jóvenes 16-29: ~30% AROPE (aproximado)")
print(f"     - Monoparental: {tasa_monoparental:.1f}% AROPE")
print("   • Riesgo estimado: ALTO")
print("   • ⚠️ NO VERIFICADO")

print("\n📌 PARA VERIFICAR ESTOS PERFILES:")
print("   1. Solicitar microdatos EPA al INE")
print("   2. Cruzar dimensiones: edad × hogar × empleo × sexo")
print("   3. Calcular tasas AROPE para combinaciones específicas")
print("   4. Ajustar por variables confusoras (región, educación, etc.)")

print("\n" + "=" * 100)

In [ ]:
# 7.3 - Limitaciones del Análisis y Trabajo Futuro
print("=" * 100)
print("⚠️ LIMITACIONES METODOLÓGICAS DEL ANÁLISIS")
print("=" * 100)

print("\n🔍 LIMITACIONES IDENTIFICADAS A LO LARGO DEL ANÁLISIS:\n")

print("=" * 100)
print("1️⃣ AUSENCIA DE ANÁLISIS MULTIDIMENSIONAL")
print("=" * 100)

print("\n❌ QUÉ NO SE PUDO HACER:")
print("-" * 100)
print("   • Cruces edad × hogar (ej: menores en hogares monoparentales)")
print("   • Cruces hogar × empleo (ej: monoparentales con adulto parado)")
print("   • Cruces edad × sexo × empleo (ej: mujeres jóvenes paradas)")
print("   • Índices de concentración multidimensional")
print("   • Perfiles de riesgo compuesto verificados con datos")

print("\n🚫 POR QUÉ NO SE PUDO HACER:")
print("-" * 100)
print("   • Tablas INE son AGREGADAS por dimensión individual")
print("   • INE_AROPE_Edad_Sexo: Solo cruza edad × sexo")
print("   • INE_AROPE_Hogar: Solo tipo de hogar (sin edad/sexo/empleo)")
print("   • INE_AROPE_Laboral: Solo situación laboral (sin edad/hogar)")
print("   • No hay tabla INE con cruces multidimensionales públicos")

print("\n⚠️ RIESGOS DE INFERIR SIN DATOS:")
print("-" * 100)
print("   • FALACIA ECOLÓGICA: Correlaciones agregadas ≠ correlaciones individuales")
print("   • Ejemplo: Alta tasa en 'Menores' + Alta tasa en 'Monoparentales'")
print("     NO implica que 'Menores monoparentales' tengan suma de riesgos")
print("   • Puede haber CONFUSORES: Región, educación, nacionalidad, etc.")
print("   • Puede haber INTERACCIONES: Efectos no aditivos entre dimensiones")

print("\n✅ SOLUCIÓN:")
print("-" * 100)
print("   • Solicitar MICRODATOS EPA al INE (previa justificación académica/oficial)")
print("   • Realizar análisis multinivel con regresión logística multivariante")
print("   • Controlar por variables confusoras")
print("   • Cuantificar interacciones entre dimensiones")

print("\n" + "=" * 100)
print("2️⃣ FALTA DE CUANTIFICACIÓN POBLACIONAL (HOGAR Y LABORAL)")
print("=" * 100)

print("\n❌ QUÉ NO SE PUDO HACER:")
print("-" * 100)
print("   • Calcular PERSONAS afectadas en situaciones laborales")
print("   • Calcular PERSONAS afectadas por tipo de hogar")
print("   • Priorizar por IMPACTO REAL (tasa × volumen)")
print("   • Comparar cambios absolutos vs relativos")

print("\n🚫 POR QUÉ NO SE PUDO HACER:")
print("-" * 100)
print("   • No existe tabla INE_Poblacion_Hogar pública")
print("   • No existe tabla INE_Poblacion_Laboral pública")
print("   • Solo disponibles: INE_Poblacion_Edad_Sexo")

print("\n⚠️ IMPLICACIÓN:")
print("-" * 100)
print("   • Análisis de HOGAR y LABORAL basado SOLO EN TASAS")
print("   • Grupo con alta tasa puede tener bajo volumen → Bajo impacto agregado")
print("   • Ejemplo: Si monoparentales son 50.3% AROPE pero solo 5% hogares,")
print("     impacto agregado menor que 'Ocupados' 15.6% si son 60% población")
print("   • NO podemos priorizar por criterio de 'mayor número de personas afectadas'")

print("\n✅ SOLUCIÓN:")
print("-" * 100)
print("   • Cruzar con EPA: Población por situación laboral")
print("   • Cruzar con Censo: Población por tipo de hogar")
print("   • Calcular: Personas afectadas = Tasa AROPE × Población del grupo")
print("   • Priorizar: Grupos con alta tasa Y alto volumen")

print("\n" + "=" * 100)
print("3️⃣ SESGOS TEMPORALES Y PERÍODOS HETEROGÉNEOS")
print("=" * 100)

print("\n⚠️ PERÍODOS DIFERENTES POR DIMENSIÓN:")
print("-" * 100)
print("   • EDAD y SEXO:    2008-2023 (16 años) - Serie COMPLETA con crisis 2008")
print("   • HOGAR:          2013-2023 (11 años) - Empieza en PICO de crisis")
print("   • LABORAL:        2014-2023 (10 años) - Empieza POST-CRISIS (recuperación)")

print("\n❌ CONSECUENCIAS:")
print("-" * 100)
print("   • Comparaciones entre dimensiones NO son estrictamente comparables")
print("   • HOGAR captura 2013 (pico) pero no shock inicial 2008-2012")
print("   • LABORAL pierde TODO el impacto crisis 2008-2013 (paro 8%→26%)")
print("   • Cambios desde 2013/2014 muestran RECUPERACIÓN, no ciclo completo")

print("\n⚠️ SESGOS EN INTERPRETACIÓN:")
print("-" * 100)
print("   • 'Reducciones desde 2013' en HOGAR → Recuperación post-pico, NO mejora estructural")
print("   • 'Estabilidad desde 2014' en LABORAL → No vemos explosión inicial de paro")
print("   • Comparar EDAD (2008+) con LABORAL (2014+) → Contextos históricos diferentes")

print("\n✅ RECOMENDACIÓN:")
print("-" * 100)
print("   • Solicitar a INE: Extender series HOGAR y LABORAL a 2008")
print("   • Si no disponible: Advertir EXPLÍCITAMENTE en conclusiones (✅ ya hecho)")
print("   • Interpretar cambios como DESCRIPTIVOS del período, no tendencias universales")

print("\n" + "=" * 100)
print("4️⃣ GRANULARIDAD LIMITADA DE CATEGORÍAS")
print("=" * 100)

print("\n⚠️ CATEGORÍAS DEMASIADO AGREGADAS:")
print("-" * 100)
print("   • EDAD: 6 grupos quinquenales/decenales → Oculta sub-tramos")
print("     - '16-29 años': Agrupa estudiantes + trabajadores jóvenes + parados")
print("   • HOGAR: Tipos amplios → No distingue número de hijos, edades")
print("     - '1 adulto + niños': No diferencia 1 vs 3+ niños, edades de niños")
print("   • LABORAL: 4 categorías → No distingue tipo de empleo")
print("     - 'Ocupados': Incluye temporales + indefinidos + autónomos + jornadas parciales")
print("     - Oculta 'working poor' (pobreza laboral por precariedad)")

print("\n❌ CONSECUENCIAS:")
print("-" * 100)
print("   • Heterogeneidad INTERNA de cada categoría no cuantificada")
print("   • Subgrupos de alto riesgo OCULTOS en promedios")
print("   • Ejemplo: Tasa 'Ocupados' 15.6% esconde que temporales pueden tener 30%+")

print("\n✅ RECOMENDACIÓN:")
print("-" * 100)
print("   • Desagregar 'Ocupados': Temporal/Indefinido, Jornada completa/Parcial")
print("   • Desagregar 'Menores': 0-5 años / 6-11 / 12-15 (vulnerabilidad diferencial)")
print("   • Desagregar 'Monoparentales': Por número y edad de niños")
print("   • Fuente: Microdatos EPA o solicitud específica a INE")

print("\n" + "=" * 100)
print("5️⃣ PROMEDIOS NO PONDERADOS (LABORAL)")
print("=" * 100)

print("\n⚠️ METODOLOGÍA LIMITADA:")
print("-" * 100)
print("   • Tabla INE_AROPE_Laboral NO tiene categoría 'Total'")
print("   • Solo tiene Sexo = 'Hombres' / 'Mujeres'")
print("   • Solución aplicada: Promedio aritmético simple")
print("     AROPE_promedio = (AROPE_Hombres + AROPE_Mujeres) / 2")

print("\n❌ PROBLEMA:")
print("-" * 100)
print("   • Promedio simple asume poblaciones H/M iguales en cada situación")
print("   • Realidad: Desbalances (ej: más hombres parados que mujeres)")
print("   • Sesgo estimado: ±1-2pp si desbalance moderado, ±2-3pp si severo")

print("\n⚠️ EJEMPLO ILUSTRATIVO:")
print("-" * 100)
print("   Parados Hombres: 60% AROPE (100,000 personas)")
print("   Parados Mujeres: 70% AROPE (50,000 personas)")
print("   • Nuestro promedio: (60+70)/2 = 65.0%")
print("   • Promedio real ponderado: (60×100k + 70×50k)/150k = 63.3%")
print("   • Error: +1.7pp")

print("\n✅ MITIGACIÓN APLICADA:")
print("-" * 100)
print("   • Limitación #5 en Sección 6: Advertencia explícita ✅")
print("   • Nota: 'Puede ocultar diferencias de género' ✅")
print("   • Ranking cualitativo se mantiene válido (orden no cambia)")
print("   • Cifras exactas tienen margen ±1-2pp")

print("\n✅ SOLUCIÓN IDEAL:")
print("-" * 100)
print("   • Cruzar con EPA: Población H/M por situación laboral")
print("   • Calcular promedio ponderado:")
print("     AROPE_ponderado = (AROPE_H × Pob_H + AROPE_M × Pob_M) / (Pob_H + Pob_M)")

print("\n" + "=" * 100)
print("6️⃣ AGREGACIÓN TERRITORIAL (NACIONAL)")
print("=" * 100)

print("\n⚠️ SOLO NIVEL NACIONAL:")
print("-" * 100)
print("   • Todo el análisis usa datos de 'España' (agregado nacional)")
print("   • NO se analizan diferencias regionales (CCAA, provincias)")

print("\n❌ CONSECUENCIAS:")
print("-" * 100)
print("   • Oculta HETEROGENEIDAD REGIONAL:")
print("     - Andalucía: ~35% AROPE")
print("     - País Vasco: ~15% AROPE")
print("     - Diferencia: 20pp (comparable a brecha edad o empleo)")
print("   • Políticas nacionales uniformes pueden ser ineficientes")
print("   • Grupos vulnerables varían por región:")
print("     - Menores más vulnerables en Sur")
print("     - Mayores más vulnerables en Galicia")

print("\n✅ RECOMENDACIÓN:")
print("-" * 100)
print("   • Consultar análisis regional: notebook 05_analisis_geografico_ccaa")
print("   • Cruzar dimensiones con territorio para políticas focalizadas")
print("   • Ejemplo: 'Menores en Andalucía' vs 'Menores en Navarra'")

print("\n" + "=" * 100)

print("\n📋 TRABAJO FUTURO RECOMENDADO:")
print("=" * 100)

print("\n🔬 INVESTIGACIÓN AVANZADA:")
print("-" * 100)
print("   1. Solicitar microdatos EPA al INE")
print("   2. Análisis multinivel: edad × sexo × hogar × empleo × región")
print("   3. Regresión logística multivariante (controlar confusores)")
print("   4. Cuantificar interacciones entre dimensiones")
print("   5. Calcular índices de concentración multidimensional")

print("\n📊 MEJORAS EN DATOS:")
print("-" * 100)
print("   1. Extender series HOGAR y LABORAL a 2008")
print("   2. Obtener datos poblacionales por hogar y situación laboral")
print("   3. Desagregar categorías amplias (Ocupados, Menores, etc.)")
print("   4. Incluir variables adicionales: educación, nacionalidad, discapacidad")

print("\n🗺️ ANÁLISIS TERRITORIAL:")
print("-" * 100)
print("   1. Replicar análisis por CCAA (17 comunidades)")
print("   2. Análisis provincial (52 provincias) si datos disponibles")
print("   3. Análisis urbano/rural")
print("   4. Identificar clusters geográficos de vulnerabilidad")

print("\n" + "=" * 100)

In [ ]:
# 7.4 - Recomendaciones de Política Pública
print("=" * 100)
print("🎯 RECOMENDACIONES DE POLÍTICA PÚBLICA")
print("=" * 100)

print("\n📌 CRITERIOS DE PRIORIZACIÓN:")
print("-" * 100)
print("   • VULNERABILIDAD: Tasa AROPE del grupo (% en riesgo)")
print("   • HETEROGENEIDAD: Coeficiente de variación de la dimensión (estratificación)")
print("   • IMPACTO: Volumen poblacional afectado (cuando disponible)")
print("   • INTERSECCIONALIDAD: Acumulación de vulnerabilidades (inferido)")

print("\n⚠️ IMPORTANTE: Recomendaciones basadas en TASAS agregadas nacionales")
print("   • Faltan datos de volumen poblacional (HOGAR, LABORAL)")
print("   • Faltan datos de cruces multidimensionales")
print("   • Faltan datos regionales (heterogeneidad territorial)")
print("   • → Recomendaciones son ORIENTATIVAS, requieren validación con datos completos")

print("\n" + "=" * 100)
print("1️⃣ PRIORIDAD MÁXIMA: INFANCIA Y HOGARES MONOPARENTALES")
print("=" * 100)

print("\n🔴 EVIDENCIA:")
print("-" * 100)
print(f"   • Menores de 16 años: {tasa_menores_16:.1f}% AROPE (2023)")
print(f"   • Hogares monoparentales: 50.3% AROPE (2023)")
print(f"   • Monoparentales = OUTLIER ESTADÍSTICO (>P75+1.5×IQR)")
print(f"   • Convergencia: Mayoría monoparentales tienen niños dependientes")

print("\n💡 POLÍTICAS RECOMENDADAS:")
print("-" * 100)
print("   A. TRANSFERENCIAS DIRECTAS:")
print("      • Incrementar cuantía Ingreso Mínimo Vital (IMV) para hogares con menores")
print("      • Complemento específico para monoparentales (actual insuficiente)")
print("      • Prestación por hijo a cargo: Aumentar y hacer universal (no solo <100€)")
print("      • Deducción fiscal progresiva (más cuantía para rentas bajas)")

print("\n   B. SERVICIOS PÚBLICOS:")
print("      • Educación infantil 0-3 años: Gratuita y universal")
print("      • Comedores escolares: Gratuitos para familias vulnerables")
print("      • Becas escolares: Material, actividades extraescolares, transporte")
print("      • Ayudas vivienda: Priorizar monoparentales con menores")

print("\n   C. CONCILIACIÓN LABORAL:")
print("      • Permisos parentales remunerados (100% salario)")
print("      • Jornadas reducidas con garantía salarial para cuidado")
print("      • Flexibilidad horaria sin penalización salarial")
print("      • Teletrabajo como derecho para familias con menores")

print("\n   D. EMPLEO PARA MONOPARENTALES:")
print("      • Bonificaciones empresariales por contratar monoparentales")
print("      • Formación profesional específica con apoyo de cuidado")
print("      • Reserva de plazas en empleo público (criterio vulnerable)")

print("\n🎯 IMPACTO ESPERADO:")
print("-" * 100)
print("   • Reducir tasa AROPE menores: 28.9% → <20% (objetivo EU 2030)")
print("   • Reducir tasa monoparentales: 50.3% → ~30% (converger a media hogares)")
print("   • Romper transmisión intergeneracional pobreza")

print("\n" + "=" * 100)
print("2️⃣ PRIORIDAD ALTA: EMPLEO Y POBREZA LABORAL")
print("=" * 100)

print("\n🔴 EVIDENCIA:")
print("-" * 100)
print("   • Parados: 62.1% AROPE (2023)")
print("   • Factor 4.0x vs Ocupados")
print("   • Coef.variación LABORAL: 67.2% (DIMENSIÓN MÁS ESTRATIFICANTE)")
print("   • Working poor: 15.6% ocupados en AROPE → Precariedad laboral")

print("\n💡 POLÍTICAS RECOMENDADAS:")
print("-" * 100)
print("   A. INSERCIÓN LABORAL (PARADOS):")
print("      • Políticas activas empleo focalizadas: Jóvenes, larga duración, >45 años")
print("      • Formación profesional dual (empresa + centro educativo)")
print("      • Orientación laboral personalizada (no masiva)")
print("      • Bonificaciones contratación grupos vulnerables")
print("      • Garantía Juvenil: Reforzar y extender >30 años si procedente")

print("\n   B. CALIDAD DEL EMPLEO (WORKING POOR):")
print("      • Subida SMI progresiva (actual ~1,134€/mes → objetivo 60% salario mediano)")
print("      • Combatir temporalidad abusiva: Inspección laboral, sanciones")
print("      • Limitar parcialidad involuntaria (derecho a jornada completa)")
print("      • Fortalecer negociación colectiva: Convenios más robustos")
print("      • In-work benefits: Complemento salarial para trabajadores con bajos ingresos")

print("\n   C. PROTECCIÓN DESEMPLEO:")
print("      • Extender duración prestaciones (actual max 24 meses)")
print("      • Aumentar cobertura (actual ~55% parados)")
print("      • Subsidio desempleo: Cuantía suficiente para salir de AROPE")
print("      • Evitar caída a IMV sin prestación intermedia")

print("\n🎯 IMPACTO ESPERADO:")
print("-" * 100)
print("   • Reducir tasa AROPE parados: 62.1% → <50%")
print("   • Reducir tasa AROPE ocupados: 15.6% → <10% (eliminar working poor)")
print("   • Aumentar tasa empleo de calidad (estable, jornada completa, salario digno)")

print("\n" + "=" * 100)
print("3️⃣ PRIORIDAD MEDIA: BRECHA DE GÉNERO E INTERSECCIONALIDAD")
print("=" * 100)

print("\n🟠 EVIDENCIA:")
print("-" * 100)
print(f"   • Brecha de género: +{brecha_genero:.1f}pp (2023)")
print("   • Coef.variación SEXO: Baja (~8%) en agregado nacional")
print("   • PERO: Brecha se amplifica en intersecciones:")
print("     - Mujeres monoparentales (mayoría son mujeres)")
print("     - Mujeres paradas (posible mayor vulnerabilidad)")
print("     - Mujeres mayores solas (pensiones más bajas)")

print("\n💡 POLÍTICAS RECOMENDADAS:")
print("-" * 100)
print("   A. IGUALDAD SALARIAL:")
print("      • Auditorías obligatorias brecha salarial en empresas")
print("      • Transparencia salarial (publicar rangos por puesto)")
print("      • Sanciones efectivas por discriminación salarial")

print("\n   B. CORRESPONSABILIDAD EN CUIDADOS:")
print("      • Permisos paternidad iguales e intransferibles (actual 16 semanas)")
print("      • Servicios públicos cuidado: Niños, mayores, dependientes")
print("      • Reparto equitativo trabajo doméstico (campañas, educación)")

print("\n   C. PROTECCIÓN VIOLENCIA DE GÉNERO:")
print("      • Ayudas específicas víctimas violencia (vivienda, empleo, ingresos)")
print("      • Garantizar independencia económica para salir de situación violenta")

print("\n   D. PENSIONES DIGNAS (MUJERES MAYORES):")
print("      • Complemento pensiones mínimas para mujeres con carreras discontinuas")
print("      • Reconocer períodos de cuidado en cálculo pensión")

print("\n🎯 IMPACTO ESPERADO:")
print("-" * 100)
print("   • Cerrar brecha de género: +1.6pp → <1pp")
print("   • Reducir vulnerabilidad mujeres en intersecciones críticas")
print("   • Mejorar pensiones mujeres mayores")

print("\n" + "=" * 100)
print("4️⃣ POLÍTICAS TRANSVERSALES")
print("=" * 100)

print("\n💡 RECOMENDACIONES APLICABLES A TODAS LAS DIMENSIONES:")
print("-" * 100)

print("\n   A. VIVIENDA:")
print("      • Parque público vivienda: Alquiler social (<30% ingresos)")
print("      • Ayudas al alquiler: Aumentar cuantía y cobertura")
print("      • Limitar precio alquiler (zonas tensionadas)")
print("      • Rehabilitación energética (reducir pobreza energética)")

print("\n   B. EDUCACIÓN:")
print("      • Gratuidad efectiva: Libros, material, comedor, actividades")
print("      • Becas universitarias: Cuantía suficiente (no solo matrícula)")
print("      • Educación compensatoria (refuerzo alumnado vulnerable)")
print("      • FP dual: Conectar educación con empleo de calidad")

print("\n   C. SANIDAD:")
print("      • Sanidad pública universal: Incluir dental, psicológica")
print("      • Copagos farmacéuticos: Exención para rentas bajas")
print("      • Prevención y promoción salud en zonas vulnerables")

print("\n   D. SERVICIOS SOCIALES:")
print("      • Reforzar red servicios sociales: Ratio trabajadores/población")
print("      • Atención personalizada (no burocracia masiva)")
print("      • Acompañamiento integral: Empleo, vivienda, salud, educación")

print("\n   E. FISCALIDAD PROGRESIVA:")
print("      • Incrementar tipos IRPF rentas altas")
print("      • Impuesto patrimonio robusto (evitar elusión)")
print("      • Combatir fraude fiscal (recuperar recursos)")
print("      • Financiar servicios públicos y transferencias")

print("\n" + "=" * 100)
print("5️⃣ MEJORAS EN DATOS Y SEGUIMIENTO")
print("=" * 100)

print("\n📊 RECOMENDACIONES AL INE:")
print("-" * 100)
print("   1. Extender series temporales:")
print("      • HOGAR: Publicar 2008-2012 (si datos existen)")
print("      • LABORAL: Publicar 2008-2013 (si datos existen)")

print("\n   2. Publicar datos poblacionales:")
print("      • Población por tipo de hogar (anual)")
print("      • Población por situación laboral (anual)")
print("      • Permitir cálculo de personas afectadas")

print("\n   3. Tablas de cruces multidimensionales:")
print("      • Edad × Hogar (menores en monoparentales)")
print("      • Hogar × Empleo (monoparentales parados)")
print("      • Edad × Sexo × Empleo (mujeres jóvenes paradas)")
print("      • Versión agregada para proteger privacidad")

print("\n   4. Microdatos:")
print("      • Facilitar acceso microdatos EPA a investigadores")
print("      • Protocolo más ágil (actual muy burocrático)")

print("\n   5. Desagregación de categorías:")
print("      • Ocupados: Temporal/Indefinido, Completo/Parcial, Sector")
print("      • Menores: 0-5, 6-11, 12-15 años")
print("      • Monoparentales: Por número y edad de hijos")

print("\n🎯 MONITOREO Y EVALUACIÓN:")
print("-" * 100)
print("   • Dashboard público AROPE: Actualización trimestral")
print("   • Evaluación ex-ante políticas: Simulación impacto sobre AROPE")
print("   • Evaluación ex-post: Medir efectividad políticas implementadas")
print("   • Informes anuales: Progreso hacia objetivos EU 2030 (reduce 15M personas)")

print("\n" + "=" * 100)
print("6️⃣ CONEXIÓN CON ANÁLISIS TERRITORIAL")
print("=" * 100)

print("\n🗺️ INTEGRACIÓN CON ANÁLISIS REGIONAL:")
print("-" * 100)
print("   • Este análisis (06_sociodemografico): Nivel NACIONAL")
print("   • Análisis complementario: 05_analisis_geografico_ccaa")
print("   • Necesidad: CRUZAR dimensiones sociodemográficas × territorio")

print("\n💡 ANÁLISIS TERRITORIAL PENDIENTE:")
print("-" * 100)
print("   • Menores por CCAA: ¿Dónde concentrar políticas infancia?")
print("   • Monoparentales por CCAA: ¿Regiones con mayor vulnerabilidad?")
print("   • Parados por CCAA: ¿Políticas empleo focalizadas regionalmente?")
print("   • Brecha género por CCAA: ¿Heterogeneidad regional?")

print("\n🎯 POLÍTICAS TERRITORIALIZADAS:")
print("-" * 100)
print("   • Adaptar cuantías transferencias a coste vida regional")
print("   • Focalizar políticas empleo en CCAA alta tasa paro")
print("   • Reforzar servicios sociales en regiones más vulnerables")
print("   • Coordinación Estado-CCAA-Ayuntamientos")

print("\n" + "=" * 100)

print("\n📋 SÍNTESIS FINAL:")
print("=" * 100)

print("\n✅ GRUPOS PRIORITARIOS IDENTIFICADOS:")
print("-" * 100)
print("   1. Menores de 16 años (28.9% AROPE)")
print("   2. Hogares monoparentales (50.3% AROPE - OUTLIER)")
print("   3. Personas paradas (62.1% AROPE)")
print("   4. Trabajadores precarios (15.6% ocupados en AROPE)")

print("\n📊 DIMENSIONES MÁS ESTRATIFICANTES:")
print("-" * 100)
print("   1. SITUACIÓN LABORAL (CV 67.2% - ALTÍSIMA)")
print("   2. TIPO DE HOGAR (CV 40.3% - ALTA)")
print("   3. EDAD (CV ~35% - MODERADA-ALTA)")
print("   4. SEXO (CV ~8% - BAJA en agregado, alta en intersecciones)")

print("\n🎯 PRINCIPIOS RECTORES:")
print("-" * 100)
print("   • FOCALIZACIÓN: Políticas diferenciadas por grupo (heterogeneidad justifica)")
print("   • UNIVERSALISMO: Servicios públicos de calidad para tod@s")
print("   • INTERSECCIONALIDAD: Abordar acumulación de vulnerabilidades")
print("   • PREVENCIÓN: Invertir en infancia para romper reproducción pobreza")
print("   • EVIDENCIA: Basar políticas en datos (requiere mejorar información)")

print("\n🚀 OBJETIVO:")
print("-" * 100)
print("   • Reducir tasa AROPE nacional: 26.0% (2023) → <15% (objetivo UE 2030)")
print("   • Reducir desigualdades intra-nacionales (entre grupos)")
print("   • Garantizar igualdad de oportunidades real")
print("   • Construir sociedad más cohesionada y justa")

print("\n" + "=" * 100)
print("✅ ANÁLISIS SOCIODEMOGRÁFICO COMPLETADO")
print("=" * 100)